
# Real-world data coding for neuroscientists (ReCoN)

### MSc in Translational Neuroscience,

### Department of Brain Sciences, Faculty of Medicine,

### Imperial College London

### Contributors: Anastasia Ilina, Cecilia Rodriguez, Marirena Bafaloukou, Katarzyna Marta Zoltowska, Rishideep Chatterjee, Sahar Rahbar, Cynthia Sandor

### Autumn 2025


# 🧠 Computing with Real‑World Medical Data in Python (MSc Translational Neuroscience)
## Tutorial 3 – Tabular data with pandas on Electronic Healthcare Records (EHR)

**Part 1: Data cleaning and preprocessing with pandas**






## Learning outcomes

**Overall aim**: develop skills for statistical analysis of longitudinal medical data.

**You will learn to:**
- Manipulate tabular data in `pandas` (indexing, filtering, grouping, reshaping).
- Pre‑clean longitudinal EHR data (duplicates, types, missingness, **data entry errors in medications**).
- Engineer features with functions and `.apply`.


## 📚 Supporting matetial - pandas cheat sheet
> This cheat sheet is a great reference for pandas methods:

> [Pandas Cheat Sheet](https://pandas.pydata.org/Pandas_Cheat_Sheet.pdf)


## 🎯 Foundations: pandas DataFrames (theory)


![pandas meme](https://i.programmerhumor.io/2025/03/bf92b0031ba27033fcf1534cd911e834c963afbc0c31b90fc9550eef0fafed92.png)

In [2]:
import pandas as pd
import numpy as np

# We'll make a tiny mock DataFrame to play with
data = {
    "patient_id": ["P01", "P02", "P03", "P04", "P01", "P02", "P03", "P04"],
    "sex": ["Male", "Female", "Female", "Male", "Male", "Female", "Female", "Male"],
    "date_of_birth": ["14.05.1950", "22.08.1965", "30.11.1970", "15.01.1980",
                                  "14.05.1950", "22.08.1965", "30.11.1970", "15.01.1980"],
    "updrs_iii": [22, 30, 40, None, 25, 31, 40, None],     # missing value
    "ledd_mg": [300, 500, 800, 0, 500, 600, 850, 0],
    "visit_date": ["2023-01-01", "2023-01-02", "2023-01-03", "2023-01-04", "2024-02-08", "2025-02-23", "2023-10-06", "2024-06-04"],
    "medication": ["Co-Carbidopa 12.5/50", "Rotigotine 0.5", "   Co-BEneldopa;", "", "Co-carbidopa,,", "Rotigotine.  ", "LEVODOPA + ENTACAPONE", "N/A"],
    "other_medication": ['Yes', 'Yes', 'No', 'No', 'Yes', 'No', 'Yes', 'No'],
    'BMI': ['22.5', '25.0', '30.0', '28.0', '22.7', '24.0', '36.0', '27.6'],
    "id_last_update": ["09/2024", "01/2025", "12/2023", "09/2024", "09/2024", "01/2025", "12/2023", "09/2024"]
}
df = pd.DataFrame(data)
df


,patient_id,sex,date_of_birth,updrs_iii,ledd_mg,visit_date,medication,other_medication,BMI,id_last_update
0,P01,Male,14.05.1950,22.0,300,2023-01-01,Co-Carbidopa 12.5/50,Yes,22.5,09/2024
1,P02,Female,22.08.1965,30.0,500,2023-01-02,Rotigotine 0.5,Yes,25.0,01/2025
2,P03,Female,30.11.1970,40.0,800,2023-01-03,Co-BEneldopa;,No,30.0,12/2023
3,P04,Male,15.01.1980,NaN,0,2023-01-04,,No,28.0,09/2024
4,P01,Male,14.05.1950,25.0,500,2024-02-08,"Co-carbidopa,,",Yes,22.7,09/2024
5,P02,Female,22.08.1965,31.0,600,2025-02-23,Rotigotine.,No,24.0,01/2025
6,P03,Female,30.11.1970,40.0,850,2023-10-06,LEVODOPA + ENTACAPONE,Yes,36.0,12/2023
7,P04,Male,15.01.1980,NaN,0,2024-06-04,N/A,No,27.6,09/2024




### 1. Selection and Filtering

**Concept:**
A **DataFrame** is a core `pandas` object — conceptually similar to an Excel sheet — but far more powerful for analysis.
It stores data in **rows** and **columns**, each with **labels** (names), and is ideal for working with structured data such as **Electronic Health Records (EHRs)**.

#### 🩺 **Why this matters in medical data**

EHRs are almost always stored in tabular form, making DataFrames a natural fit for biomedical and clinical research workflows.

* Each **row** typically represents a single **observation** or **encounter** — for example, a patient’s visit to a clinic.
* Each **column** represents a different **attribute** or **feature** of that record — such as patient ID, age, diagnosis, treatment, or visit date.

In our **synthetic Parkinson’s dataset**,
each row represents a **medication record** for a patient during one clinic visit.
Because each patient can have **multiple visits**, you will see several rows per patient, recorded at different time points.

#### 🧩 **What’s inside this dataset**

Alongside demographic information (e.g., **age at visit**, **sex**), the dataset includes several **clinical features**:

* **UPDRS III** — a clinical rating scale assessing the severity of motor symptoms in Parkinson’s disease.
* **LEDD (Levodopa Equivalent Daily Dose)** — a calculated metric expressing the total daily dose of dopaminergic medication, normalized across different drugs.

These variables will be used throughout the tutorial for cleaning, transformation, visualization, and statistical modeling.


#### 🔍 **Selecting and filtering in `pandas`**

You can access or subset DataFrames using **labels** (row/column names) or **logical conditions**:

* `df.loc[rows, columns]` → select by explicit labels
* `df.iloc[row_indices, col_indices]` → select by position (numeric index)
* `df.query("condition")` → filter rows using SQL-like syntax

This allows you to quickly focus on, for example:

* All visits for a particular patient
* Only those observations with a specific diagnosis or medication
* A subset of columns containing clinical or demographic features



In [3]:
# Let's get a quick summary of the DataFrame first: its structure, data types, and non-null counts per column
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8 entries, 0 to 7
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   patient_id        8 non-null      object 
 1   sex               8 non-null      object 
 2   date_of_birth     8 non-null      object 
 3   updrs_iii         6 non-null      float64
 4   ledd_mg           8 non-null      int64  
 5   visit_date        8 non-null      object 
 6   medication        8 non-null      object 
 7   other_medication  8 non-null      object 
 8   BMI               8 non-null      object 
 9   id_last_update    8 non-null      object 
dtypes: float64(1), int64(1), object(8)
memory usage: 772.0+ bytes


In [4]:
# You can use .head() and .tail() to quickly inspect the first or last few rows of a DataFrame
df.head()  # shows first 5 rows by default

,patient_id,sex,date_of_birth,updrs_iii,ledd_mg,visit_date,medication,other_medication,BMI,id_last_update
0,P01,Male,14.05.1950,22.0,300,2023-01-01,Co-Carbidopa 12.5/50,Yes,22.5,09/2024
1,P02,Female,22.08.1965,30.0,500,2023-01-02,Rotigotine 0.5,Yes,25.0,01/2025
2,P03,Female,30.11.1970,40.0,800,2023-01-03,Co-BEneldopa;,No,30.0,12/2023
3,P04,Male,15.01.1980,NaN,0,2023-01-04,,No,28.0,09/2024
4,P01,Male,14.05.1950,25.0,500,2024-02-08,"Co-carbidopa,,",Yes,22.7,09/2024


In [5]:
df.tail(2)  # last 2 rows

,patient_id,sex,date_of_birth,updrs_iii,ledd_mg,visit_date,medication,other_medication,BMI,id_last_update
6,P03,Female,30.11.1970,40.0,850,2023-10-06,LEVODOPA + ENTACAPONE,Yes,36.0,12/2023
7,P04,Male,15.01.1980,NaN,0,2024-06-04,N/A,No,27.6,09/2024


In [6]:
# Select columns
df[["patient_id", "visit_date", "updrs_iii"]]

,patient_id,visit_date,updrs_iii
0,P01,2023-01-01,22.0
1,P02,2023-01-02,30.0
2,P03,2023-01-03,40.0
3,P04,2023-01-04,NaN
4,P01,2024-02-08,25.0
5,P02,2025-02-23,31.0
6,P03,2023-10-06,40.0
7,P04,2024-06-04,NaN


In [7]:
# you can overlay it with .head() or .tail()
df[["patient_id", "visit_date", "updrs_iii"]].head()

,patient_id,visit_date,updrs_iii
0,P01,2023-01-01,22.0
1,P02,2023-01-02,30.0
2,P03,2023-01-03,40.0
3,P04,2023-01-04,NaN
4,P01,2024-02-08,25.0


In [8]:
# more ways to select columns
cols = ["patient_id", "visit_date", "updrs_iii"]
df[cols]


,patient_id,visit_date,updrs_iii
0,P01,2023-01-01,22.0
1,P02,2023-01-02,30.0
2,P03,2023-01-03,40.0
3,P04,2023-01-04,NaN
4,P01,2024-02-08,25.0
5,P02,2025-02-23,31.0
6,P03,2023-10-06,40.0
7,P04,2024-06-04,NaN


In [9]:
# Row and column selection by label
df.loc[1, "sex"]                   # row with label 1 (might not be the first row) and the column named "sex"

'Female'

In [10]:
# Row and column selection by index position
df.iloc[0, 1]                       # first row (position 0) and the second column (position 1)

'Male'

In [11]:

# Select multiple rows/cols by label
df.loc[0:2, ["patient_id", "visit_date"]]

,patient_id,visit_date
0,P01,2023-01-01
1,P02,2023-01-02
2,P03,2023-01-03


In [12]:

# Conditional selection: only patients with UPDRS score lower than 30 
df.loc[df["updrs_iii"] < 30, ["patient_id", "visit_date", "ledd_mg", "updrs_iii"]]

,patient_id,visit_date,ledd_mg,updrs_iii
0,P01,2023-01-01,300,22.0
4,P01,2024-02-08,500,25.0


In [13]:
# Conditional selection: only male patients
df.loc[df["sex"] == 'Male', ["patient_id",  "date_of_birth", "ledd_mg", "updrs_iii", "sex"]] 
# you can also use !=, >, <, >=, <=

,patient_id,date_of_birth,ledd_mg,updrs_iii,sex
0,P01,14.05.1950,300,22.0,Male
3,P04,15.01.1980,0,NaN,Male
4,P01,14.05.1950,500,25.0,Male
7,P04,15.01.1980,0,NaN,Male


In [14]:
# Conditional selection: patients on medication
df.loc[df["medication"] != "None", ["patient_id", "visit_date", "medication"]]

,patient_id,visit_date,medication
0,P01,2023-01-01,Co-Carbidopa 12.5/50
1,P02,2023-01-02,Rotigotine 0.5
2,P03,2023-01-03,Co-BEneldopa;
3,P04,2023-01-04,
4,P01,2024-02-08,"Co-carbidopa,,"
5,P02,2025-02-23,Rotigotine.
6,P03,2023-10-06,LEVODOPA + ENTACAPONE
7,P04,2024-06-04,N/A


In [15]:
# you can also select columns using regex of the column names 

# let's select the columns that contain "date"
df.filter(regex="date")


,date_of_birth,visit_date,id_last_update
0,14.05.1950,2023-01-01,09/2024
1,22.08.1965,2023-01-02,01/2025
2,30.11.1970,2023-01-03,12/2023
3,15.01.1980,2023-01-04,09/2024
4,14.05.1950,2024-02-08,09/2024
5,22.08.1965,2025-02-23,01/2025
6,30.11.1970,2023-10-06,12/2023
7,15.01.1980,2024-06-04,09/2024


In [16]:

# not lets select the columns that end with "id"
df.filter(regex="id$")  # $ means "ends with"

,patient_id
0,P01
1,P02
2,P03
3,P04
4,P01
5,P02
6,P03
7,P04


In [17]:
# now let's select the columns that start with "id"
df.filter(regex="^id")  # ^ means "starts with"



,id_last_update
0,09/2024
1,01/2025
2,12/2023
3,09/2024
4,09/2024
5,01/2025
6,12/2023
7,09/2024


In [18]:
# you can also combine multiple conditions using & (and), | (or), ~ (not)
# Note: use parentheses around each condition when combining


# let's select the column that starts with "id" and ends with "date"
df.filter(regex="^id.*date$")

,id_last_update
0,09/2024
1,01/2025
2,12/2023
3,09/2024
4,09/2024
5,01/2025
6,12/2023
7,09/2024


In [19]:
# you can overlay different methods as much as you want
# let's select the columns that contain "date" and only show the first 3 rows
df.loc[df["sex"] == 'Male', ["patient_id",  "date_of_birth", "ledd_mg", "updrs_iii", "sex"]].filter(regex = "date").head(1)

,date_of_birth
0,14.05.1950


In [20]:
# you can also call the columns using df.filter().columns
df.loc[df["ledd_mg"] < 300, df.filter(regex = "date").columns]

,date_of_birth,visit_date,id_last_update
3,15.01.1980,2023-01-04,09/2024
7,15.01.1980,2024-06-04,09/2024


In [21]:
# For those familiar with SQL, you can also use query() method for conditional selection
# Using query() (SQL-like)
df.query("updrs_iii < 30 and sex == 'Male'")

,patient_id,sex,date_of_birth,updrs_iii,ledd_mg,visit_date,medication,other_medication,BMI,id_last_update
0,P01,Male,14.05.1950,22.0,300,2023-01-01,Co-Carbidopa 12.5/50,Yes,22.5,09/2024
4,P01,Male,14.05.1950,25.0,500,2024-02-08,"Co-carbidopa,,",Yes,22.7,09/2024



### 🍎 <span style="color:red; font-weight:bold">Student challenge 


 <span style="color:red; font-weight:bold">Now your turn!</span>

  <span style="color:red; font-weight:bold"> Use the dataset `df` to pratice filtering.</span>

**Tasks:**

1. Write code to select all rows for `patient_id` "P02" and only the columns `visit_date`, `updrs_iii`, and `ledd_mg`. 
2. Write code to select show me the last 3 rows for columns `patient_id`, `visit_date`, and `medication` using first .tail() and then DO IT AGAIN but using .loc[]
3. Can you find which gender is the patient with zero `ledd_mg` using a query?
4. For all female patients, display all the columns that contain "medication" in their name.


In [22]:
df.loc[df["patient_id"] == "P02", ["visit_date", "updrs_iii", "ledd_mg"]]

,visit_date,updrs_iii,ledd_mg
1,2023-01-02,30.0,500
5,2025-02-23,31.0,600


In [23]:
# Write your code for .tail()
df.tail(3)[["patient_id", "visit_date", "medication"]]

# Write your code for .loc[]
df.loc[df.index[-3:], ["patient_id", "visit_date", "medication"]]

,patient_id,visit_date,medication
5,P02,2025-02-23,Rotigotine.
6,P03,2023-10-06,LEVODOPA + ENTACAPONE
7,P04,2024-06-04,N/A


In [24]:
df.query("ledd_mg == 0")[["patient_id", "sex"]]

,patient_id,sex
3,P04,Male
7,P04,Male


In [25]:
df.loc[df["sex"] == 'Female', df.filter(regex="medication").columns]

,medication,other_medication
1,Rotigotine 0.5,Yes
2,Co-BEneldopa;,No
5,Rotigotine.,No
6,LEVODOPA + ENTACAPONE,Yes


## 🧩 1️⃣ Understanding Data Types in pandas

### Why it matters

In EHR data, each column stores a **different type of information**:

* **Numeric:** clinical scores (e.g. `updrs_iii`, `ledd_mg`)
* **Categorical:** sex, medication type
* **Datetime:** visit or birth dates
* **String/object:** free text (e.g., clinician notes)

Getting types right lets pandas apply the right operations (e.g., compute means for numbers, sort by date, count categories, etc.).


### Let's check your data types!

In [26]:
df.dtypes

patient_id           object
sex                  object
date_of_birth        object
updrs_iii           float64
ledd_mg               int64
visit_date           object
medication           object
other_medication     object
BMI                  object
id_last_update       object
dtype: object

> This provides you with a quick overview of how pandas “understands” each column.

### 🧮 1️⃣ Numeric types — `int64` and `float64`

* **What they are:** Whole numbers (`int64`) or decimals (`float64`).
* **Used for:** Quantitative, measurable values that you can perform arithmetic on (add, subtract, divide, compute means, etc.).
* **Examples in your dataset:**

  * `ledd_mg`: medication dose in mg (integer or float)
  * `updrs_iii`: motor severity score (float because it contains `NaN` missing values)

> 💡 **Note:** In pandas, if any value in a numeric column is missing (`NaN`), pandas automatically upgrades the column to `float64`, since `NaN` is a floating-point concept.

**Typical operations:**


In [27]:
print("Average dose (mg):", df["ledd_mg"].mean())   # average dose

Average dose (mg): 443.75


Check if BMI is numeric! If not, convert it and verify by running `df.dtypes` again.



In [28]:
df["BMI"] = pd.to_numeric(df["BMI"], errors="coerce")  # convert BMI to numeric, coerce errors to NaN

In [29]:
df["BMI"] 

0    22.5
1    25.0
2    30.0
3    28.0
4    22.7
5    24.0
6    36.0
7    27.6
Name: BMI, dtype: float64

In [30]:
df["updrs_iii"].max()    # highest severity

np.float64(40.0)

In [31]:
df["ledd_mg"] / 1000     # convert mg to g

0    0.30
1    0.50
2    0.80
3    0.00
4    0.50
5    0.60
6    0.85
7    0.00
Name: ledd_mg, dtype: float64

### Let's dive into the available operations a bit deeper!

#### 🔢 Basic arithmetic operations on numeric columns

Imagine we have a dataframe with clinical scores and medication doses just for one patient:

```python
df = pd.DataFrame({
    "patient_id": ["P01"] * 5,
    "age": [65, 65, 66, 66, 67],
    "BMI": [22.53, 22.28, 22.67, 24.02, 24.72],
    "visit_date": pd.to_datetime(["2024-01-15", "2024-06-20", 
                                "2024-12-05", "2025-03-10", 
                                "2025-09-15"]),
    "updrs_iii": [22.0, 25.5, 30.0, 28.0, 35.0],
    "ledd_mg": [300, 400, 500, 600, 700]
})
```

Pandas supports a variety of arithmetic operations that can help us with analysis!

| Operation      | Example                       | Meaning                               |
| -------------- | ----------------------------- | ------------------------------------- |
| Addition       | `df["updrs_iii"] + 5`         | Increase all UPDRS scores by 5 points |
| Subtraction    | `df["age"] - 60`              | Difference from 60 years old          |
| Multiplication | `df["ledd_mg"] * 0.001`       | Convert mg to g                       |
| Division       | `df["ledd_mg"] / df["BMI"]`   | LEDD normalised by BMI                |
| Power          | `df["age"] ** 2`              | Square of age                         |
| Rounding       | `df["BMI"].round(1)`          | Round to 1 decimal                    |
| Absolute value | `df["updrs_iii"].abs()`       | Remove negative sign (if any)         |
| Cumulative sum | `df["ledd_mg"].cumsum()`      | Running total of LEDD exposure                       |
| Difference     | `df["ledd_mg"].diff()`        | Change between two visits             |
| Ranking        | `df["updrs_iii"].rank()`      | Patient rank by severity              |
| Sorting        | `df.sort_values("updrs_iii")` | Order by score                        |

> 💡 These work directly on pandas columns because they’re implemented **vector-wise** — the operation applies to each cell automatically.

---


#### 📈 Descriptive statistics (summary measures)

pandas makes it easy to calculate common summary statistics that describe a numeric column’s distribution.

| Statistic                   | Code                                          | Meaning                                                               |
| --------------------------- | --------------------------------------------- | --------------------------------------------------------------------- |
| **Mean** (average)          | `df["updrs_iii"].mean()`                      | The *centre* of the data — total divided by number of observations.   |
| **Median**                  | `df["updrs_iii"].median()`                    | The *middle* value when sorted; less sensitive to outliers.           |
| **Mode**                    | `df["updrs_iii"].mode()`                      | The *most frequent* value in the column.                              |
| **Standard deviation (SD)** | `df["updrs_iii"].std()`                       | Average distance of each value from the mean — a measure of *spread*. |
| **Variance**                | `df["updrs_iii"].var()`                       | SD squared; another measure of spread.                                |
| **Minimum / Maximum**       | `df["updrs_iii"].min()`, `.max()`             | Smallest and largest values.                                          |
| **Count (non-missing)**     | `df["updrs_iii"].count()`                     | Number of non-NaN values.                                             |
| **Sum**                     | `df["updrs_iii"].sum()`                       | Total of all values.                                                  |
| **Quantiles**               | `df["updrs_iii"].quantile([0.25, 0.5, 0.75])` | 25th, 50th (median), and 75th percentiles.                            |
| **Skewness**                | `df["updrs_iii"].skew()`                      | Whether the distribution is lopsided (tail to right or left).         |
| **Kurtosis**                | `df["updrs_iii"].kurt()`                      | How “peaked” or “flat” the distribution is.                           |


![mean_mode_median](https://tutor2u-net.imgix.net/subjects/geography/downloads/livestreams/studynoteimages/mean-median-mode.png?auto=compress%2Cformat&fit=clip&q=80&w=800)



> 📊 You can view many of these at once with:


In [32]:
df[["updrs_iii", "ledd_mg"]].describe()

,updrs_iii,ledd_mg
count,6.000000,8.000000
mean,31.333333,443.750000
std,7.474401,324.518875
min,22.000000,0.000000
25%,26.250000,225.000000
50%,30.500000,500.000000
75%,37.750000,650.000000
max,40.000000,850.000000


![mean_mode_median_meme](images/IMG_2884.WEBP)


---
### 🧠 Quick explanations of key statistical terms
---
#### Mean (Average)

The arithmetic mean is the **sum of all values divided by the number of observations**.

$
\text{Mean} = \frac{x_1 + x_2 + ... + x_n}{n}
$

Example:
If LEDD doses are 300, 500, and 700 mg → mean = (300 + 500 + 700)/3 = 500 mg.

Used when data are **symmetrically distributed**.

---

#### Median

The **middle** value once the data are sorted.

In  both discrete and continuous data, it works like this: if there are 9 patients, the median is the 5th when sorted by UPDRS score. For even counts, it’s the average of the two middle values.


✅ **Resistant to outliers**, so it’s better than the mean for skewed data.

---

#### Mode

The **most frequent** value in the data.
Useful for categorical or discrete numeric data (e.g., common UPDRS III test score, most frequent clinic).

---

#### Standard Deviation (SD)

Measures **spread or variability** — how far data points tend to deviate from the mean.

$
\text{SD} = \sqrt{\frac{\sum (x_i - \bar{x})^2}{n-1}}
$

* Low SD → most values close to the mean.
* High SD → data widely spread out.

In EHRs, SD helps gauge **heterogeneity** (e.g., how variable LEDD doses are between patients).

![](https://mathbitsnotebook.com/Algebra2/Statistics/normalpercentilegrph2.jpg)
---

#### Variance

The average of squared deviations from the mean.

$
\text{Variance} = \text{SD}^2
$

> Used mainly for statistical modelling; less intuitive than SD because it’s in squared units.

---

#### Quantiles & Percentiles

Divide data into equal parts:

* 25th percentile (Q1): 25 % of data below it
* 50th percentile (Q2): median
* 75th percentile (Q3): 75 % of data below it
* IQR = Q3 − Q1 measures spread of the middle 50 %.

![](https://i.sstatic.net/dTDGD.png)

---

#### Range

Maximum minus minimum — simplest measure of spread.

$
\text{Range} = \text{max} - \text{min}
$

---

#### Skewness

Indicates whether the data are **asymmetric**:

* **Right-skewed** (positive): tail to the right (e.g., LEDD — most low, few very high).
* **Left-skewed** (negative): tail to the left.

In discrete data, you can estimate skewness as Pearson's first coefficient of skewness (mode skewness):
$\text{Skewness} = \frac{\text{Mean} - \text{Mode}}{\text{SD}}$

In continuous data, you can quantify skewness as Pearson's second coefficient of skewness (median skewness):
$\text{Skewness} = \frac{3(\text{Mean} - \text{Median})}{\text{SD}}$    

![](https://www.statisticshowto.com/wp-content/uploads/2014/02/pearson-mode-skewness.jpg)
---
#### Kurtosis

Describes how “peaked” or “flat” the distribution is compared to a normal curve as well as how 'fat' the tails of the distribution are. High kurtosis → heavy tails (more outliers).

![](https://miro.medium.com/v2/resize:fit:1400/format:webp/1*JbVwjNfpkplE9G8Dnosqgw.jpeg )








### 🍎 <span style="color:red; font-weight:bold"> Student challenge  </span>

<span style="color:red; font-weight:bold">  Now, your turn! </span>

Calculate the mean, median, mode, standard deviation, variance, min, max, count, sum, 25th, 50th, and 75th percentiles, skew and kurtosis  for the `updrs_iii` column in the dataframe `df`. 

In [33]:
## Write your code here:

print("Mean UPDRS III:", df["updrs_iii"].mean())
print("Median UPDRS III:", df["updrs_iii"].median())
print("Mode UPDRS III:", df["updrs_iii"].mode()[0])  # mode() returns a Series
print("Standard Deviation UPDRS III:", df["updrs_iii"].std())
print("Variance UPDRS III:", df["updrs_iii"].var())
print("Min UPDRS III:", df["updrs_iii"].min())
print("Max UPDRS III:", df["updrs_iii"].max())
print("Count UPDRS III:", df["updrs_iii"].count())
print("Sum UPDRS III:", df["updrs_iii"].sum())
print("25th Percentile UPDRS III:", df["updrs_iii"].quantile(0.25))
print("50th Percentile UPDRS III:", df["updrs_iii"].quantile(0.50))
print("75th Percentile UPDRS III:", df["updrs_iii"].quantile(0.75))     
print("Skewness UPDRS III:", df["updrs_iii"].skew())
print("Kurtosis UPDRS III:", df["updrs_iii"].kurtosis())    



Mean UPDRS III: 31.333333333333332
Median UPDRS III: 30.5
Mode UPDRS III: 40.0
Standard Deviation UPDRS III: 7.474400756359446
Variance UPDRS III: 55.86666666666666
Min UPDRS III: 22.0
Max UPDRS III: 40.0
Count UPDRS III: 6
Sum UPDRS III: 188.0
25th Percentile UPDRS III: 26.25
50th Percentile UPDRS III: 30.5
75th Percentile UPDRS III: 37.75
Skewness UPDRS III: 0.16699800620194719
Kurtosis UPDRS III: -1.655877444307107


#### <span style="color:red"> 🍎 Bonus question! </span>

![](images/IMG_2876.JPG)

* Which of those cats have **higher standard deviation** than the other? Why?

* Which one has the **higher kurtosis**? Why? 

### 🔢 Combining numeric operations

You can combine methods easily:

In [34]:
# Example summaries
mean_updrs = df["updrs_iii"].mean()
std_updrs = df["updrs_iii"].std()
cv_updrs = std_updrs / mean_updrs   # Coefficient of variation (relative spread)

print(f"Mean: {mean_updrs:.1f}, SD: {std_updrs:.1f}, CV: {cv_updrs:.2f}")

Mean: 31.3, SD: 7.5, CV: 0.24


In [35]:
# You can implement this using a single line!


# OR using a single line
cv_updrs = df["updrs_iii"].std() / df["updrs_iii"].mean()

print(f"CV: {cv_updrs:.2f}")

CV: 0.24



### 🧩 Creating New Columns and Rows in pandas

Once you understand **numeric data types** (`int64`, `float64`), you can start **building new variables** from existing ones — just like you’d do in Excel formulas, but faster and more reproducibly.

#### 🧮 1️⃣ Creating new columns

You can create a new column simply by assigning to a name inside square brackets `[]`.
```python

df["new_column_name"] = <some expression>
```
This adds a new column to your DataFrame.
The expression can be:

* A **constant** (e.g., `0`, `np.nan`, `1.5`)
* An **operation** on existing columns (addition, subtraction, division, etc.)
* A **function** applied to other columns



#### ➕ Add or subtract values

Let’s start with your existing numeric columns:
`ledd_mg`, `updrs_iii` 



In [36]:
# Increase all LEDD doses by 100 mg
df["ledd_plus_100"] = df["ledd_mg"] + 100

# Difference between UPDRS score and 30
df["updrs_diff"] = df["updrs_iii"] - 30

In [37]:
df[['ledd_mg','ledd_plus_100', 'updrs_iii', 'updrs_diff']]

,ledd_mg,ledd_plus_100,updrs_iii,updrs_diff
0,300,400,22.0,-8.0
1,500,600,30.0,0.0
2,800,900,40.0,10.0
3,0,100,NaN,NaN
4,500,600,25.0,-5.0
5,600,700,31.0,1.0
6,850,950,40.0,10.0
7,0,100,NaN,NaN


#### ✖️ Multiply or divide columns

In [38]:
# Convert LEDD from mg to grams
df["ledd_g"] = df["ledd_mg"] / 1000

# LEDD normalised by BMI (dose per body mass)
df["ledd_per_bmi"] = df["ledd_mg"] / df["BMI"]

df[['ledd_mg','ledd_g', 'BMI', 'ledd_per_bmi']]

,ledd_mg,ledd_g,BMI,ledd_per_bmi
0,300,0.30,22.5,13.333333
1,500,0.50,25.0,20.000000
2,800,0.80,30.0,26.666667
3,0,0.00,28.0,0.000000
4,500,0.50,22.7,22.026432
5,600,0.60,24.0,25.000000
6,850,0.85,36.0,23.611111
7,0,0.00,27.6,0.000000


#### ⚙️ Combine multiple operations

In [39]:
# Adjust LEDD based on both BMI and severity
df["dose_index"] = (df["ledd_mg"] / df["BMI"]) * df["updrs_iii"]



> 💡 **Note:** pandas applies arithmetic “vectorised” — the operation happens automatically on every row.


### 🧩 3️⃣ Conditional columns (using `np.where` or boolean logic)

You can create new variables based on **conditions** — for example, flagging patients on medication or severe disease.

In [40]:
import numpy as np

# 1 = on medication, 0 = no medication
df["on_medication"] = np.where(df["ledd_mg"] > 0, 1, 0)

# Flag severe UPDRS scores (≥ 59)
df["is_severe"] = np.where(df["updrs_iii"] >= 59, 1, 0)


You can also combine multiple conditions using `&` (AND), `|` (OR), and `~` (NOT):

In [41]:
df["pd_like"] = np.where(
    (df["ledd_mg"] > 0) & (df["updrs_iii"] > 0),
    "Likely_PD",
    "Unlikely_PD"
)

###  4️⃣ Deleting columns
You can delete columns using `df.drop(columns=[...])` or `del df['column_name']`.


In [42]:
# let's look at the columns we have in the dataframe
df.columns

Index(['patient_id', 'sex', 'date_of_birth', 'updrs_iii', 'ledd_mg',
       'visit_date', 'medication', 'other_medication', 'BMI', 'id_last_update',
       'ledd_plus_100', 'updrs_diff', 'ledd_g', 'ledd_per_bmi', 'dose_index',
       'on_medication', 'is_severe', 'pd_like'],
      dtype='object')

In [43]:
# let's create a list of columns to delete (all of the newly created ones)
columns_to_delete = ["ledd_plus_100", "updrs_diff", "ledd_g", "ledd_per_bmi", "dose_index", "on_medication", "is_severe", "pd_like"]

# delete the columns using df.drop()
df = df.drop(columns=columns_to_delete)

In [44]:
df.columns 

Index(['patient_id', 'sex', 'date_of_birth', 'updrs_iii', 'ledd_mg',
       'visit_date', 'medication', 'other_medication', 'BMI',
       'id_last_update'],
      dtype='object')

### ➕ 5️⃣ Adding new rows

You can append a single observation (row) using `pd.concat()` or `DataFrame.loc[]`.

In [45]:
# Example: add one new patient visit (dictionary of values)
new_row = {
    "patient_id": "P05",
    "sex": "Male",
    "ledd_mg": 550,
    "updrs_iii": 42,
    "BMI": 26.4,
    "date_of_birth": "10.10.1960",
    "visit_date": "2024-05-01",
    "medication": "Entacapone 200",
}
df = pd.concat([df, pd.DataFrame([new_row])], ignore_index=True)

Or modify a specific row directly:

In [46]:
df.loc[len(df)] = ["P06", "Unidentified", "22.05.1980", 28, 400, "2025-01-05", "Levodopa", "No", 23.4, "09/2025"]

> ⚠️ **Tip:** Avoid manual row addition in real analyses — in EHR data, rows usually come from merging or concatenating datasets.







### 🧠 6️⃣ Common numeric column operations for new columns

| Operation       | Code                        | Description                        |
| --------------- | --------------------------- | ---------------------------------- |
| Addition        | `df["A"] + df["B"]`         | Sum two columns                    |
| Subtraction     | `df["A"] - df["B"]`         | Difference between columns         |
| Multiplication  | `df["A"] * 10`              | Scale values                       |
| Division        | `df["A"] / df["B"]`         | Create ratio                       |
| Modulus         | `df["A"] % 2`               | Remainder after division           |
| Power           | `df["A"] ** 2`              | Square or cube values              |
| Logarithm       | `np.log(df["A"])`           | Useful for skewed data (e.g. LEDD) |
| Rounding        | `df["A"].round(1)`          | Round numeric results              |
| Type conversion | `df["A"].astype("float64")` | Change numeric type                |

---



### 🍎 <span style="color:red; font-weight:bold">Student challenge 


 <span style="color:red; font-weight:bold">Now your turn!</span>

  <span style="color:red; font-weight:bold"> Use the dataset `df` to explore numeric data and create a few new variables.</span>

**Tasks:**

1. Calculate the **mean**, **median**, and **standard deviation** of `ledd_mg`. 

2. Compare the **mean** and **median** — does this suggest LEDD is *skewed*? Is it right-skewed or left-skewed? Get a value for **skewness** using both Pearson's formula and `.skew()`.

3. Compute the **interquartile range (IQR)** for `updrs_iii` (Q3 − Q1) as a variable `iqr_updrs`.

4. Create a new column `ledd_per_updrs` = LEDD ÷ UPDRS III.

5. Create a Boolean column `high_bmi` that is `True` if BMI > 30 (obesity threshold).

6. One way to deal with the skewness of LEDD is to standardise it using a ** modified z-score**. In comparison to conventional z-score, which measures how far the value is from the mean in terms of standard deviations, modified z-score uses median and median absolute deviation (MAD), making it usable in skewed distributions.
Create a new column `ledd_zscore` using the formula: 

    $ ledd\_zscore = 0.6745 \times \frac{ledd\_mg - median(ledd\_mg)}{MAD(ledd\_mg)}$

    where $MAD(ledd\_mg) = median(|ledd\_mg - median(ledd\_mg)|)$
    
    (Hint: use and `np.abs()` functions)






In [47]:
# Calculate the mean, median and standard deviation of LEDD 
mean_ledd = df["ledd_mg"].mean(skipna=True)
median_ledd = df["ledd_mg"].median(skipna=True)
std_ledd = df["ledd_mg"].std(skipna=True)
print(f"Mean LEDD: {mean_ledd:.1f}, Median LEDD: {median_ledd:.1f}, SD LEDD: {std_ledd:.1f}")


Mean LEDD: 450.0, Median LEDD: 500.0, SD LEDD: 288.7


In [48]:
# Apply pearson's second coefficient of skewness
skewness_ledd = 3 * (mean_ledd - median_ledd) / std_ledd
print(f"Skewness of LEDD using Pearson\'s: {skewness_ledd:.2f}")

# Calculate it using .skew()
print(f"Skewness of LEDD (using .skew()): {df['ledd_mg'].skew():.2f}")

Skewness of LEDD using Pearson's: -0.52
Skewness of LEDD (using .skew()): -0.43


In [49]:
# Compute the interquartile range (IQR) of UPDRS III
iqr_updrs = df["updrs_iii"].quantile(0.75) - df["updrs_iii"].quantile(0.25)
print(f"IQR of UPDRS III: {iqr_updrs:.1f}")

IQR of UPDRS III: 12.8


In [50]:
# Create a new column ledd_per_updrs
df["ledd_per_updrs"] = df["ledd_mg"] / df["updrs_iii"]
df[['ledd_mg', 'updrs_iii', 'ledd_per_updrs']]


,ledd_mg,updrs_iii,ledd_per_updrs
0,300,22.0,13.636364
1,500,30.0,16.666667
2,800,40.0,20.000000
3,0,NaN,NaN
4,500,25.0,20.000000
5,600,31.0,19.354839
6,850,40.0,21.250000
7,0,NaN,NaN
8,550,42.0,13.095238
9,400,28.0,14.285714


In [51]:
# Create a new column flagging high BMI (≥ 30)
df["high_bmi"] = np.where(df["BMI"] >= 30, 1, 0)
df[['BMI', 'high_bmi']] 

,BMI,high_bmi
0,22.5,0
1,25.0,0
2,30.0,1
3,28.0,0
4,22.7,0
5,24.0,0
6,36.0,1
7,27.6,0
8,26.4,0
9,23.4,0


In [52]:
# Standardize LEDD by creating a z-score column

# calculate MAD of LEDD
mad_ledd = (df["ledd_mg"] - df["ledd_mg"].median()).abs().median()

df["ledd_zscore"] = 0.6745 * (df["ledd_mg"] - df["ledd_mg"].median()) / mad_ledd

df[['ledd_mg', 'ledd_zscore']]

,ledd_mg,ledd_zscore
0,300,-0.899333
1,500,0.000000
2,800,1.349000
3,0,-2.248333
4,500,0.000000
5,600,0.449667
6,850,1.573833
7,0,-2.248333
8,550,0.224833
9,400,-0.449667


## Other types of data structures in pandas


### 🧩 2️⃣ Boolean type — `bool`

We've already encountered Boolean columns when we created `high_bmi` above.
* **What it is:** Logical True/False values.
* **Used for:** Flags or binary indicators (e.g., has medication, abnormal result).
* **Examples:**
  You can *create* Boolean columns:



In [53]:
df["is_on_medication"] = df["ledd_mg"] > 0

  This column will be `True` where dose > 0 and `False` otherwise.

In [54]:
df[['ledd_mg', 'is_on_medication']]

,ledd_mg,is_on_medication
0,300,True
1,500,True
2,800,True
3,0,False
4,500,True
5,600,True
6,850,True
7,0,False
8,550,True
9,400,True


### 🔤  Object / String type — `object` or `string`

* **What it is:** Free text or mixed data that pandas can’t classify automatically.
* **Used for:** Patient identifiers, notes, or descriptive text.
* **Examples:**

  * `patient_id`
  * `medication` (if not converted to category)




> ⚠️ `object` columns are flexible but slower for analysis — convert them to `category` or numeric if possible.

We can use methods such as `.unique()`, `.nunique()` and `.value_counts()` to explore unique values in an Object/String column:

In [55]:
df["medication"].unique()

array(['Co-Carbidopa 12.5/50', 'Rotigotine 0.5', '   Co-BEneldopa;', '',
       'Co-carbidopa,,', 'Rotigotine.  ', 'LEVODOPA + ENTACAPONE', 'N/A',
       'Entacapone 200', 'Levodopa'], dtype=object)

In [56]:
df["medication"].nunique()            # count unique drug types

10

In [57]:
df["medication"].value_counts()   # count occurrences of each drug type

medication
Co-Carbidopa 12.5/50     1
Rotigotine 0.5           1
   Co-BEneldopa;         1
                         1
Co-carbidopa,,           1
Rotigotine.              1
LEVODOPA + ENTACAPONE    1
N/A                      1
Entacapone 200           1
Levodopa                 1
Name: count, dtype: int64


> ⚠️ As you can see, we have 2 **occurences of Co-carbidopa** and 2 occurences of **Rotigotine**, but due to inconsistent capitalization and spacing, pandas treats them as **different values**. This is a common issue in EHR data entry, and it is important to **clean** such data before analysis using string methods.

#### 🔤 Common String Operations in pandas (object / string dtype)


Because a pandas column is a **Series** (a whole column, not a single string), we use the special `.str` accessor to apply text operations **element-wise** (to each cell).

> For example:


In [58]:
df["medication"].str.lower()  

0     co-carbidopa 12.5/50
1           rotigotine 0.5
2            co-beneldopa;
3                         
4           co-carbidopa,,
5            rotigotine.  
6    levodopa + entacapone
7                      n/a
8           entacapone 200
9                 levodopa
Name: medication, dtype: object

#### 🧮 Common `.str` methods and when to use them

| Method                                  | What it does                               | When to use                                                       |
| --------------------------------------- |  ------------------------------------------ | ----------------------------------------------------------------- |
| `.str.lower()`                          | Converts all text to lowercase             | Standardise case before comparison (`"Levodopa"` vs `"levodopa"`) |
| `.str.upper()`                          | Converts all letters to uppercase          | Formatting or matching uppercase codes                            |
| `.str.title()`                          | Capitalises first letter of each word      | Clean messy labels for presentation                               |
| `.str.strip()`                          | Removes leading/trailing spaces            | Clean extra whitespace from imported data                         |
| `.str.contains("levodopa", case=False)` |  Returns True if text contains substring    | Filter rows with certain keywords (e.g., `"Levodopa"`)            |
| `.str.replace("Co-", "")`               |  Replaces text patterns (supports regex)    | Clean prefixes/suffixes (e.g., “Co-”)                             |
| `.str.startswith("Levo")`               |  True/False if string starts with a pattern | Identify all Levodopa derivatives                                 |
| `.str.endswith("dopa")`                 |  True/False if ends with a pattern          | Detect variants of a drug ending in “dopa”                        |
| `.str.len()`                            |  Returns string length                      | Check length of IDs or text fields                                |
| `.str.slice(0,3)`                       |  Extract substring by position              | Get first 3 characters of `patient_id`                            |
| `.str.split(" ")`                       |  Splits a string into a list by spaces      | Separate multi-word fields (e.g., "Levodopa COMT")                |
| `.str.replace(r"\d+", "", regex=True)`  |  Remove all numbers using regex             | Clean IDs, strip version numbers                                  |
| `.str.cat(sep=", ")`                    |  Concatenate strings in a column            | Join text values for summary outputs                              |
| `.str.get(0)`                           |  Get character at a position                | Extract first letter of patient code                              |
| `.str.findall("pattern")`               | Return all regex matches per row           | Advanced pattern extraction (e.g., dose values from notes)        |

---

#### 🧠 Why use `.str`?

* Each cell is a string, but the column is a **pandas Series** — `.str` lets you broadcast string methods to *every row* at once.
* Without `.str`, normal Python string functions (like `.lower()`) won’t work on a whole column.


In [59]:
# Find all patients on any dopa derivative formulation
mask = df["medication"].str.contains("dopa", case=False, na=False)
df[mask][["patient_id", "medication"]]

,patient_id,medication
0,P01,Co-Carbidopa 12.5/50
2,P03,Co-BEneldopa;
4,P01,"Co-carbidopa,,"
6,P03,LEVODOPA + ENTACAPONE
9,P06,Levodopa



### 🍎 <span style="color:red; font-weight:bold">Student challenge 


 <span style="color:red; font-weight:bold">Now your turn!</span>

  <span style="color:red; font-weight:bold"> Pre-clean the dataset and explore medical prescription data.</span>

**Tasks:**

1. Using the `.str` methods, standardize the text in the `medication` column so that:
   - All entries are in lowercase.
   - Leading and trailing spaces are removed.
   - Any occurrence of "co-" is removed from the medication names.
   - Missing values are consistently represented as ""

2. Perform a new `.value_counts()` on the cleaned `medication` column to see how many unique medications there are and their frequencies.

3. Create new boolean column  'on_dopa_derivative' and 'on_dopamine_agonist' to mark patients on those medications 

4. Create new boolean column 'on_combination_therapy' to mark patients two or more medications prescribed (e.g. COMT with Dopa Derivative)


In [60]:
# Standardize medication names by stripping whitespace and punctuation, and converting to lowercase
df["medication_clean"] = df["medication"].str.replace('N/A', '').str.strip(' .,/;!0123456789').str.lower().replace('co-', '')
df[["medication", "medication_clean"]]

,medication,medication_clean
0,Co-Carbidopa 12.5/50,co-carbidopa
1,Rotigotine 0.5,rotigotine
2,Co-BEneldopa;,co-beneldopa
3,,
4,"Co-carbidopa,,",co-carbidopa
5,Rotigotine.,rotigotine
6,LEVODOPA + ENTACAPONE,levodopa + entacapone
7,N/A,
8,Entacapone 200,entacapone
9,Levodopa,levodopa


In [61]:
df['medication_clean'].value_counts()

medication_clean
co-carbidopa             2
rotigotine               2
                         2
co-beneldopa             1
levodopa + entacapone    1
entacapone               1
levodopa                 1
Name: count, dtype: int64

In [62]:
df['on_dopa_derivative'] = df['medication_clean'].str.contains('dopa', na=False)
df[['medication_clean', 'on_dopa_derivative']]

,medication_clean,on_dopa_derivative
0,co-carbidopa,True
1,rotigotine,False
2,co-beneldopa,True
3,,False
4,co-carbidopa,True
5,rotigotine,False
6,levodopa + entacapone,True
7,,False
8,entacapone,False
9,levodopa,True


In [63]:
df["on_dopamine_agonist"] = df["medication_clean"].str.startswith("rotigotine", na=False)
df[["medication_clean", "on_dopamine_agonist"]]

,medication_clean,on_dopamine_agonist
0,co-carbidopa,False
1,rotigotine,True
2,co-beneldopa,False
3,,False
4,co-carbidopa,False
5,rotigotine,True
6,levodopa + entacapone,False
7,,False
8,entacapone,False
9,levodopa,False


In [64]:
df["on_combination_therapy"] = df["medication_clean"].str.contains("+", regex=False, na=False)

df[["medication_clean", "on_combination_therapy"]]

,medication_clean,on_combination_therapy
0,co-carbidopa,False
1,rotigotine,False
2,co-beneldopa,False
3,,False
4,co-carbidopa,False
5,rotigotine,False
6,levodopa + entacapone,True
7,,False
8,entacapone,False
9,levodopa,False


In [65]:
# or another way to escape regex special character
df["on_combination_therapy"] = df["medication_clean"].str.contains(r"\+", na=False)
df[["medication_clean", "on_combination_therapy"]]

,medication_clean,on_combination_therapy
0,co-carbidopa,False
1,rotigotine,False
2,co-beneldopa,False
3,,False
4,co-carbidopa,False
5,rotigotine,False
6,levodopa + entacapone,True
7,,False
8,entacapone,False
9,levodopa,False



#### 💡 Tips

If you have messy text data:

1. Clean and standardise with `.str.lower()` and `.str.strip()`.
2. `.str.strip()` can also remove specific characters like punctuation or numbers, e.g.
    ```python
    df["column"].str.strip(".,;!0123")
    ```
2. Use `.str.contains()` or `.str.startswith()` for filtering.
3. Convert cleaned columns to `category` if values repeat often.
4. You can chain multiple `.str` methods together for more complex cleaning, e.g.:
    ```python
    df["column"].str.lower().str.strip().str.replace("old", "new")
    ```


### 🧬 4️⃣ Categorical types — `category`

* **What they are:** Columns that contain a limited number of possible values.
* **Used for:** Grouping and statistical comparisons — they are memory-efficient and improve performance.
* **Examples:**

  * `sex`: “Male”, “Female”
  * `other_medication`: “Yes”, “No”
  * `medication`: drug names (repeated categories)

You can convert to category with:

In [66]:
df["sex"] = df["sex"].astype("category")

In [67]:
df["sex"]

0            Male
1          Female
2          Female
3            Male
4            Male
5          Female
6          Female
7            Male
8            Male
9    Unidentified
Name: sex, dtype: category
Categories (3, object): ['Female', 'Male', 'Unidentified']

**Why use `category`:**

* Less memory use for repeated labels
* Helpful when modelling (encoded automatically as dummy variables)

### `groupby()` for categorical data and other types
You can use `.groupby()` to **aggregate** numeric data by categories.
For example, to compute the mean UPDRS III score by sex:


In [68]:
# groupby() for UPDRS III by sex
df.groupby("sex")["updrs_iii"].mean()

/var/folders/nb/mdcnqz1d7673k0zpfm275hrm0000gp/T/ipykernel_65315/309082024.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df.groupby("sex")["updrs_iii"].mean()


sex
Female          35.250000
Male            29.666667
Unidentified    28.000000
Name: updrs_iii, dtype: float64

The `.groupby()` method doesn't require the grouping column to be of type `category`, but converting it can speed up operations and reduce memory usage, especially with large datasets.

Let's try it on other_medication, which is currently of type object (string):


In [69]:
df.groupby("other_medication")["updrs_iii"].mean()

other_medication
No     33.00
Yes    29.25
Name: updrs_iii, dtype: float64

You can also group by multiple columns, e.g., mean UPDRS III by sex and medication status:


In [70]:
df.groupby(["sex", "other_medication"])["updrs_iii"].mean()

/var/folders/nb/mdcnqz1d7673k0zpfm275hrm0000gp/T/ipykernel_65315/1652840924.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df.groupby(["sex", "other_medication"])["updrs_iii"].mean()


sex           other_medication
Female        No                  35.5
              Yes                 35.0
Male          No                   NaN
              Yes                 23.5
Unidentified  No                  28.0
              Yes                  NaN
Name: updrs_iii, dtype: float64


### 📅 3️⃣ Datetime type — `datetime64[ns]`

* **What it is:** Timestamps that include date (and optionally time) information.
* **Used for:** Age calculation, sorting visits, computing time differences, or visualising trends over time.
* **Examples:**

  * `date_of_birth`, `visit_date`, `id_last_update`

**Typical operations:**


In [71]:
# turn the visit_date column to datetime
df["visit_date"] = pd.to_datetime(df["visit_date"])
df.dtypes

patient_id                        object
sex                             category
date_of_birth                     object
updrs_iii                        float64
ledd_mg                            int64
visit_date                datetime64[ns]
medication                        object
other_medication                  object
BMI                              float64
id_last_update                    object
ledd_per_updrs                   float64
high_bmi                           int64
ledd_zscore                      float64
is_on_medication                    bool
medication_clean                  object
on_dopa_derivative                  bool
on_dopamine_agonist                 bool
on_combination_therapy              bool
dtype: object

After converting date columns with `pd.to_datetime()`, you can perform many **time-based operations** to calculate differences, extract components, or resample over time.

In [72]:
# Extract year or month
df["visit_year"] = df["visit_date"].dt.year
df["visit_month"] = df["visit_date"].dt.month

As you might remember, the 'id_last_update' is also a date, but it was logged in a different format. 
> `pd.to_datetime()` can parse many date formats automatically, but sometimes you need to **specify the format explicitly** using the `format` parameter. For example, if the date is in the format "MM/YYYY", you would use:


In [73]:

df["id_last_update"] = pd.to_datetime(df["id_last_update"], format="%m/%Y")

In [74]:

# Difference between two dates (today and last update)
df["days_since_update"] = (pd.Timestamp.now() - df["id_last_update"]).dt.days
df["days_since_update"]

0    435.0
1    313.0
2    710.0
3    435.0
4    435.0
5    313.0
6    710.0
7    435.0
8      NaN
9     70.0
Name: days_since_update, dtype: float64

In [75]:
# Difference between two dates (today and last update) in months
df["months_since_update"] = (pd.Timestamp.now() - df["id_last_update"]).dt.days / 30
df["months_since_update"]

0    14.500000
1    10.433333
2    23.666667
3    14.500000
4    14.500000
5    10.433333
6    23.666667
7    14.500000
8          NaN
9     2.333333
Name: months_since_update, dtype: float64

In [76]:
# imagine we found out that the update dates were actually recorded 2 months, 28 days and 35 minutes later than they actually were

days_update = 2 * 30 + 28
df["id_last_update"] = df["id_last_update"] - pd.Timedelta(days=days_update, hours=0, minutes=35)

In [77]:
df["id_last_update"]

0   2024-06-04 23:25:00
1   2024-10-04 23:25:00
2   2023-09-03 23:25:00
3   2024-06-04 23:25:00
4   2024-06-04 23:25:00
5   2024-10-04 23:25:00
6   2023-09-03 23:25:00
7   2024-06-04 23:25:00
8                   NaT
9   2025-06-04 23:25:00
Name: id_last_update, dtype: datetime64[ns]

In [78]:
# Difference between two dates (today and last update) in years
df["years_since_update"] = (pd.Timestamp.now() - df["id_last_update"]).dt.days / 365.25
df["years_since_update"]

0    1.431896
1    1.097878
2    2.184805
3    1.431896
4    1.431896
5    1.097878
6    2.184805
7    1.431896
8         NaN
9    0.432580
Name: years_since_update, dtype: float64



### ⏱️ Common Time & Date Operations in pandas

Here are some more examples of useful date/time operations you can perform in pandas:

| Operation                                    | Example code                                                                                     | Description                                                 | Typical use in clinical / longitudinal data          |
| -------------------------------------------- | ------------------------------------------------------------------------------------------------ | ----------------------------------------------------------- | ---------------------------------------------------- |
| **Days since last update**                   | `df["days_since_update"] = (pd.Timestamp.now() - df["id_last_update"]).dt.days`                  | Calculates number of days between the last update and today | Monitor data recency or stale patient records        |
| **Days between two events**                  | `df["days_between_visits"] = (df["visit_date"] - df["id_last_update"]).dt.days`                  | Difference between two date columns                         | Measure follow-up intervals or delays in visits      |
| **Age at visit (years)**                     | `df["age_at_visit"] = ((df["visit_date"] - df["date_of_birth"]).dt.days / 365.25).round(1)`      | Convert date difference to years                            | Calculate patient’s age dynamically at each visit    |
| **Extract year, month, day**                 | `df["visit_year"] = df["visit_date"].dt.year`<br>`df["visit_month"] = df["visit_date"].dt.month` | Extract components of the date                              | Group analyses by calendar year or seasonality       |
| **Weekday or weekend**                       | `df["visit_weekday"] = df["visit_date"].dt.day_name()`                                           | Returns full day name (Monday–Sunday)                       | Identify clinic patterns by day of the week          |
| **Time elapsed in months**                   | `df["months_since_dx"] = (df["years_since_dx"] * 12).round(0)`                                   | Converts years to months (if stored numerically)            | Harmonise units for monthly summaries                |
| **First and last recorded date per patient** | `df.groupby("patient_id")["visit_date"].agg(["min", "max"])`                                     | Finds each patient’s first and last visit date              | Identify earliest baseline and most recent follow-up |
| **Duration of follow-up (days)**             | `df["followup_duration"] = (df["max_visit"] - df["min_visit"]).dt.days`                          | Subtract first from last date                               | Measure longitudinal follow-up time                  |
| **Check for missing or future dates**        | `df.loc[df["visit_date"] > pd.Timestamp.now()]`                                                  | Find any visit recorded in the future                       | Data quality check for incorrect entry               |
| **Resampling by time period**                | `df.set_index("visit_date").resample("M")["ledd_mg"].mean()`                                     | Averages LEDD by month (resampling time series)             | Explore monthly trends in medication load            |
| **Rolling time windows**                     | `df.set_index("visit_date")["updrs_iii"].rolling("180D").mean()`                                 | Moving average over a 180-day window                        | Smooth longitudinal symptom trends                   |

---




### 💡 Tips 

* `.dt` is the **datetime accessor**, similar to `.str` for strings — it lets you access date parts and perform operations column-wise.
* `pd.Timestamp.now()` or `pd.Timestamp.today()` gives the current date/time.
* Differences between two `datetime64` columns produce a `timedelta64` series, from which you can extract `.dt.days`, etc.
* Always check for **time zone consistency** (`.dt.tz_localize()` / `.dt.tz_convert()`) in real-world datasets.

### ⚙️ 6️⃣ Missing data type — `NaN` (Not a Number)

* **What it is:** Placeholder for missing or undefined values.
* **Used for:** Representing blanks in numeric or datetime columns.
* **Handled by:** `pandas.isna()` or `.fillna()` for imputation later.

**Example:**

In [79]:
df["updrs_iii"].isna().sum()   # count missing scores

np.int64(2)



```python

```

---

### 📊 Summary Table

| pandas dtype        | Description         | Typical use                            | Example column             |
| ------------------- | ------------------- | -------------------------------------- | -------------------------- |
| `int64`             | Integer numbers     | Counts, doses                          |  `updrs_iii`                 |
| `float64`           | Decimal numbers     | Scores with decimals or missing values |   `ledd_mg`              |
| `category`          | Repeated labels     | Sex, medication type                   | `sex`  |
| `datetime64[ns]`    | Date & time         | Visits, birthdates                     | `visit_date`               |
| `object` / `string` | Text                | IDs, free text                         | `patient_id`, `medication` |
| `bool`              | True/False          | Flags, conditions                      | Derived columns            |
| `NaN`               | Missing placeholder | Incomplete data                        | Gaps in `updrs_iii`        |



## ⚙️ Understanding `.transform()` and `lambda x:` in pandas

### 🧠 Why we need `.transform()`

When you use `groupby()`, pandas usually gives you **one value per group** — for example, the minimal LEDD per patient:



In [80]:
df.groupby("patient_id")["ledd_mg"].min()

patient_id
P01    300
P02    500
P03    800
P04      0
P05    550
P06    400
Name: ledd_mg, dtype: int64


But sometimes we want to **transform** the group data **and return the result for every row** in the original DataFrame.

👉 That’s what **`.transform()`** does:
it performs a group-wise calculation but **keeps the same shape** as the original data.

#### 🔍 Example 1 — mean LEDD per patient, returned for every visit


In [81]:
df["mean_ledd_per_patient"] = (
    df.groupby("patient_id")["ledd_mg"]
      .transform("min")
)
df[["patient_id", "ledd_mg", "mean_ledd_per_patient"]]

,patient_id,ledd_mg,mean_ledd_per_patient
0,P01,300,300
1,P02,500,500
2,P03,800,800
3,P04,0,0
4,P01,500,300
5,P02,600,500
6,P03,850,800
7,P04,0,0
8,P05,550,550
9,P06,400,400



> Each visit now shows the patient’s own minimal LEDD.

> Compare this to `.groupby().min()`, which collapses the dataset to one row per patient.

This is useful when you want to **compare each observation to a group-level statistic** (e.g., how does current LEDD compare to the patient's smallest dose across observations?). We can easily compute this as a new column:

In [82]:
df["ledd_now_vs_min"] = (
    df["ledd_mg"] - df.groupby("patient_id")["ledd_mg"].transform("min")
)
df[["patient_id", "ledd_mg", "mean_ledd_per_patient", "ledd_now_vs_min"]]

,patient_id,ledd_mg,mean_ledd_per_patient,ledd_now_vs_min
0,P01,300,300,0
1,P02,500,500,0
2,P03,800,800,0
3,P04,0,0,0
4,P01,500,300,200
5,P02,600,500,100
6,P03,850,800,50
7,P04,0,0,0
8,P05,550,550,0
9,P06,400,400,0


#### 🧮 Example 2 — using custom functions inside `.transform()`
You can also pass a **custom function** instead of a predefined string like `"mean"` or `"sum"`.

In [83]:
def mad_zscore(series):
    # calculate MAD of LEDD
    mad_value = (series - series.median()).abs().median()
    return 0.6745 * (series - series.median()) / mad_value



df["ledd_mad_zscore_per_patient"] = (
    df.groupby("patient_id")["ledd_mg"].transform(mad_zscore)
)

df[["patient_id", "ledd_mg", "ledd_mad_zscore_per_patient"]]


,patient_id,ledd_mg,ledd_mad_zscore_per_patient
0,P01,300,-0.6745
1,P02,500,-0.6745
2,P03,800,-0.6745
3,P04,0,NaN
4,P01,500,0.6745
5,P02,600,0.6745
6,P03,850,0.6745
7,P04,0,NaN
8,P05,550,NaN
9,P06,400,NaN


> Here we standardise LEDD *within each patient* — the output keeps the same number of rows as the original dataset.


#### 🧩 What is `lambda x:` ?

A `lambda` is a **short, one-line function** used when you don’t want to define a full `def` block.

```python
lambda x: <expression using x>
```

* `x` represents the Series (the group of values passed in by pandas).
* Whatever the expression returns replaces the column values.

#### 💡 Example 3 — simple `lambda` transformations

In [84]:
# Center LEDD values around the median
df["ledd_centered"] = df["ledd_mg"].transform(lambda x: x - x.median())

> You can also use logical conditions inside transform to filter out values to be transformed.

In [85]:
# Add 10 to every LEDD value above 0
df["ledd_plus_10"] = df["ledd_mg"].transform(lambda x: x + 10 if x > 0 else x)

df[["patient_id", "ledd_mg", "ledd_plus_10"]]

,patient_id,ledd_mg,ledd_plus_10
0,P01,300,310
1,P02,500,510
2,P03,800,810
3,P04,0,0
4,P01,500,510
5,P02,600,610
6,P03,850,860
7,P04,0,0
8,P05,550,560
9,P06,400,410


In [86]:
# Equivalently, you can have used
df["ledd_plus_10"] = df["ledd_mg"].transform(lambda x: 0 if x <= 0 else x + 10)

df[["patient_id", "ledd_mg", "ledd_plus_10"]]

,patient_id,ledd_mg,ledd_plus_10
0,P01,300,310
1,P02,500,510
2,P03,800,810
3,P04,0,0
4,P01,500,510
5,P02,600,610
6,P03,850,860
7,P04,0,0
8,P05,550,560
9,P06,400,410


When used with `groupby`, the `lambda` receives each group separately:

In [87]:
df["ledd_centered_by_patient"] = (
    df.groupby("patient_id")["ledd_mg"]
      .transform(lambda x: x - x.median())
)
df[["patient_id", "ledd_mg", "ledd_centered_by_patient", "ledd_centered"]]

,patient_id,ledd_mg,ledd_centered_by_patient,ledd_centered
0,P01,300,-100.0,-200.0
1,P02,500,-50.0,0.0
2,P03,800,-25.0,300.0
3,P04,0,0.0,-500.0
4,P01,500,100.0,0.0
5,P02,600,50.0,100.0
6,P03,850,25.0,350.0
7,P04,0,0.0,-500.0
8,P05,550,0.0,50.0
9,P06,400,0.0,-100.0


> In this case, the **centred LEDD for each patient** - `ledd_centred_by_patient` is calculated using the **patient-specific median**. The `ledd_centered`, on the other hand, uses the median group-wise median of **all patients**.


### ⚙️ Advanced Group Operations: `.transform()` vs `.agg()`

So far, you’ve learned that:

* `.groupby()` lets us split the data into groups (e.g., by `patient_id` or `sex`).
* `.transform()` applies an operation to each group **but keeps the same number of rows** as the original DataFrame.
* `lambda x:` lets us define custom mini-functions inline.

Now we’ll explore **`.agg()` (aggregate)** — another powerful pandas method for summarising data *by group*.


#### 🧠 What’s the difference?

| Method         | Output shape                    | Typical use                                                                           |
| -------------- | ------------------------------- | ------------------------------------------------------------------------------------- |
| `.transform()` | same number of rows as original | Create new columns (e.g., patient-level means or z-scores broadcast back to each row) |
| `.agg()`       | one row per group               | Create *summary tables* (e.g., mean, SD, min, max per patient)                        |



#### 🧩 Example 1 — Using `.agg()` for summary statistics per patient

In [88]:
patient_summary = (
    df.groupby("patient_id")
      .agg({
          "ledd_mg": ["mean", "std", "min", "max"],
          "updrs_iii": ["mean", "min", "max", "count"]
      })
)
patient_summary.head()

ledd_mg                       updrs_iii                  
              mean         std  min  max      mean   min   max count
patient_id                                                          
P01          400.0  141.421356  300  500      23.5  22.0  25.0     2
P02          550.0   70.710678  500  600      30.5  30.0  31.0     2
P03          825.0   35.355339  800  850      40.0  40.0  40.0     2
P04            0.0    0.000000    0    0       NaN   NaN   NaN     0
P05          550.0         NaN  550  550      42.0  42.0  42.0     1

This creates a **multi-index column** (hierarchical header) with your chosen statistics.

✅ Each row = **one patient**
✅ Columns = **summary measures** for that patient

> 💡 You can flatten column names if you prefer simple labels:

In [89]:
patient_summary.columns = ['_'.join(col).strip() for col in patient_summary.columns.values]
patient_summary.reset_index(inplace=True)


In [90]:
patient_summary.head()

,patient_id,ledd_mg_mean,ledd_mg_std,ledd_mg_min,ledd_mg_max,updrs_iii_mean,updrs_iii_min,updrs_iii_max,updrs_iii_count
0,P01,400.0,141.421356,300,500,23.5,22.0,25.0,2
1,P02,550.0,70.710678,500,600,30.5,30.0,31.0,2
2,P03,825.0,35.355339,800,850,40.0,40.0,40.0,2
3,P04,0.0,0.000000,0,0,NaN,NaN,NaN,0
4,P05,550.0,NaN,550,550,42.0,42.0,42.0,1


#### 🧮 Example 2 — Applying multiple custom functions with `.agg()`

You can mix built-in functions, lambdas, and your own custom ones:

In [91]:
def iqr(x):
    return x.quantile(0.75) - x.quantile(0.25)

patient_summary = (
    df.groupby("patient_id")
      .agg({
          "ledd_mg": ["mean", "std", iqr],
          "updrs_iii": [lambda x: x.mean(skipna=True), "count"]
      })
)

patient_summary.head()

ledd_mg                     updrs_iii      
              mean         std    iqr <lambda_0> count
patient_id                                            
P01          400.0  141.421356  100.0       23.5     2
P02          550.0   70.710678   50.0       30.5     2
P03          825.0   35.355339   25.0       40.0     2
P04            0.0    0.000000    0.0        NaN     0
P05          550.0         NaN    0.0       42.0     1

> The `lambda x:` syntax lets you define one-line operations inline without writing a full `def` block.



#### ⚕️ Example 3 — Compare `.transform()` vs `.agg()`


In [92]:
# .transform(): returns one value per row
df["mean_ledd_per_patient"] = df.groupby("patient_id")["ledd_mg"].transform("mean")
df[['patient_id', 'ledd_mg', 'mean_ledd_per_patient']]

,patient_id,ledd_mg,mean_ledd_per_patient
0,P01,300,400.0
1,P02,500,550.0
2,P03,800,825.0
3,P04,0,0.0
4,P01,500,400.0
5,P02,600,550.0
6,P03,850,825.0
7,P04,0,0.0
8,P05,550,550.0
9,P06,400,400.0


In [93]:

# .agg(): returns one row per patient
mean_ledd_per_patient = df.groupby("patient_id")["ledd_mg"].agg("mean").reset_index()
mean_ledd_per_patient

,patient_id,ledd_mg
0,P01,400.0
1,P02,550.0
2,P03,825.0
3,P04,0.0
4,P05,550.0
5,P06,400.0


| Operation            | Returns                                 | Purpose                                      |
| -------------------- | --------------------------------------- | -------------------------------------------- |
| `.transform("mean")` | Same size as original DataFrame         | Adds a new column in the same table          |
| `.agg("mean")`       | Condensed DataFrame (one row per group) | Creates a summary table for further analysis |

---

#### 🔗 Example 4 — Merge the summary back into the main dataset

If you later want to use these statistics alongside the original data (for example, to compare each visit’s LEDD to the patient’s mean), you can merge them back on `patient_id`. 

P.S. we will cover more on merging and joining datasets in the later section of this tutorial!

In [94]:
patient_summary.columns = ['_'.join(col).strip() for col in patient_summary.columns.values]
patient_summary.reset_index(inplace=True)

In [95]:

df_new = df.merge(patient_summary, on="patient_id", how="left")
df_new

,patient_id,sex,date_of_birth,updrs_iii,ledd_mg,visit_date,medication,other_medication,BMI,id_last_update,...,ledd_now_vs_min,ledd_mad_zscore_per_patient,ledd_centered,ledd_plus_10,ledd_centered_by_patient,ledd_mg_mean,ledd_mg_std,ledd_mg_iqr,updrs_iii_<lambda_0>,updrs_iii_count
0,P01,Male,14.05.1950,22.0,300,2023-01-01,Co-Carbidopa 12.5/50,Yes,22.5,2024-06-04 23:25:00,...,0,-0.6745,-200.0,310,-100.0,400.0,141.421356,100.0,23.5,2
1,P02,Female,22.08.1965,30.0,500,2023-01-02,Rotigotine 0.5,Yes,25.0,2024-10-04 23:25:00,...,0,-0.6745,0.0,510,-50.0,550.0,70.710678,50.0,30.5,2
2,P03,Female,30.11.1970,40.0,800,2023-01-03,Co-BEneldopa;,No,30.0,2023-09-03 23:25:00,...,0,-0.6745,300.0,810,-25.0,825.0,35.355339,25.0,40.0,2
3,P04,Male,15.01.1980,NaN,0,2023-01-04,,No,28.0,2024-06-04 23:25:00,...,0,NaN,-500.0,0,0.0,0.0,0.000000,0.0,NaN,0
4,P01,Male,14.05.1950,25.0,500,2024-02-08,"Co-carbidopa,,",Yes,22.7,2024-06-04 23:25:00,...,200,0.6745,0.0,510,100.0,400.0,141.421356,100.0,23.5,2
5,P02,Female,22.08.1965,31.0,600,2025-02-23,Rotigotine.,No,24.0,2024-10-04 23:25:00,...,100,0.6745,100.0,610,50.0,550.0,70.710678,50.0,30.5,2
6,P03,Female,30.11.1970,40.0,850,2023-10-06,LEVODOPA + ENTACAPONE,Yes,36.0,2023-09-03 23:25:00,...,50,0.6745,350.0,860,25.0,825.0,35.355339,25.0,40.0,2
7,P04,Male,15.01.1980,NaN,0,2024-06-04,N/A,No,27.6,2024-06-04 23:25:00,...,0,NaN,-500.0,0,0.0,0.0,0.000000,0.0,NaN,0
8,P05,Male,10.10.1960,42.0,550,2024-05-01,Entacapone 200,NaN,26.4,NaT,...,0,NaN,50.0,560,0.0,550.0,NaN,0.0,42.0,1
9,P06,Unidentified,22.05.1980,28.0,400,2025-01-05,Levodopa,No,23.4,2025-06-04 23:25:00,...,0,NaN,-100.0,410,0.0,400.0,NaN,0.0,28.0,1




### 💡 Teaching tip

Use `.transform()` when you want to **add new columns** to the same DataFrame (same number of rows).
Use `.agg()` when you want to **create a separate summary DataFrame** (fewer rows).
Later in your course, these same operations become the basis for **statistical summaries**, **longitudinal modelling inputs**, and **data visualisation by group**.

### 🧩 Using `.apply()` in pandas

The `.apply()` method is one of pandas’ most flexible tools.
It lets you **run a custom function** on each element, column, or row of a DataFrame or Series.

### 🧠 What `.apply()` does

| Situation                         | Method                   | What it does                                            | Typical use                                     |
| --------------------------------- | ------------------------ | ------------------------------------------------------- | ----------------------------------------------- |
| **Series.apply(func)**            | `df["col"].apply(func)`  | Applies a function to **each value in a single column** | Clean or classify text (e.g., medication names) |
| **DataFrame.apply(func, axis=0)** | `df.apply(func)`         | Applies a function to each **column**                   | Compute column-wise stats                       |
| **DataFrame.apply(func, axis=1)** | `df.apply(func, axis=1)` | Applies a function to each **row**                      | Combine values from multiple columns            |


> 💡 Think of `.apply()` as a “bridge” between pandas and Python — it lets you use your own logic on pandas data.

#### 🔍 Example 1 — simple numeric function


In [96]:
df["updrs_squared"] = df["updrs_iii"].apply(lambda x: x**2)
df[["updrs_iii", "updrs_squared"]].head()

,updrs_iii,updrs_squared
0,22.0,484.0
1,30.0,900.0
2,40.0,1600.0
3,NaN,NaN
4,25.0,625.0



Each value of `updrs_iii` is passed into the function `lambda x: x**2` and replaced with the result.

### 🔍 Example 2 — applying a custom function to text data

In [97]:
def simplify_med_name(med):
    if pd.isna(med) or med == "":
        return "None"
    return med.lower().strip()

df["medication_simple_cleaning"] = df["medication"].apply(simplify_med_name)
df[["medication", "medication_simple_cleaning"]]

,medication,medication_simple_cleaning
0,Co-Carbidopa 12.5/50,co-carbidopa 12.5/50
1,Rotigotine 0.5,rotigotine 0.5
2,Co-BEneldopa;,co-beneldopa;
3,,None
4,"Co-carbidopa,,","co-carbidopa,,"
5,Rotigotine.,rotigotine.
6,LEVODOPA + ENTACAPONE,levodopa + entacapone
7,N/A,n/a
8,Entacapone 200,entacapone 200
9,Levodopa,levodopa



This runs your function on every value in the `medication` column — a **row-wise clean-up** step.

#### 🔢 Example 3 — using `.apply()` across rows

In [98]:
def ledd_per_bmi(row):
    if pd.isna(row["ledd_mg"]) or pd.isna(row["BMI"]) or row["BMI"] == 0:
        return np.nan
    return row["ledd_mg"] / row["BMI"]

df["ledd_per_bmi"] = df.apply(ledd_per_bmi, axis=1)
df[["ledd_mg", "BMI", "ledd_per_bmi"]]

,ledd_mg,BMI,ledd_per_bmi
0,300,22.5,13.333333
1,500,25.0,20.000000
2,800,30.0,26.666667
3,0,28.0,0.000000
4,500,22.7,22.026432
5,600,24.0,25.000000
6,850,36.0,23.611111
7,0,27.6,0.000000
8,550,26.4,20.833333
9,400,23.4,17.094017


> 🧩 `axis=1` means “apply this function to each **row**”;
> each row is passed to your function as a small Series (`row["colname"]`).

### 🧩 Using `.map()` in pandas

The `.map()` method applies a **simple transformation** to each element of a pandas **Series** (i.e., one column).
It’s perfect for **value-by-value replacements** or simple logic, but less flexible than `.apply()`.

#### 🧠 When to use `.map()`

| Method        | Works on               | Purpose                                            |
| ------------- | ---------------------- | -------------------------------------------------- |
| `.map()`      | Series (single column) | Quick element-wise transformation or recoding      |
| `.apply()`    | Series or DataFrame    | More complex logic using custom functions          |
| `.applymap()` | Entire DataFrame       | Apply element-wise transformation across all cells |

> 💡 `.map()` is ideal when you need to **relabel**, **replace**, or **re-encode** column values using a mapping dictionary or a simple function.


#### 🔍 Example 1 — Re-label categorical values with a dictionary

In [99]:
# Current column
df["sex"].head()

0      Male
1    Female
2    Female
3      Male
4      Male
Name: sex, dtype: category
Categories (3, object): ['Female', 'Male', 'Unidentified']

In [100]:
# Map old values to new ones
sex_map = {"Male": "M", "Female": "F"}
df["sex_short"] = df["sex"].map(sex_map)



In [101]:
df[["sex", "sex_short"]]

,sex,sex_short
0,Male,M
1,Female,F
2,Female,F
3,Male,M
4,Male,M
5,Female,F
6,Female,F
7,Male,M
8,Male,M
9,Unidentified,NaN



✅ **Result:**
Each value in `df["sex"]` is replaced with its mapped label.
Unmapped values become `NaN` (so always include all expected keys).


#### 🔍 Example 2 — Re-encode Yes/No answers numerically

In [102]:
yn_map = {"Yes": 1, "No": 0}
df["other_medication_code"] = df["other_medication"].map(yn_map)
df[["other_medication", "other_medication_code"]].head()

,other_medication,other_medication_code
0,Yes,1.0
1,Yes,1.0
2,No,0.0
3,No,0.0
4,Yes,1.0


✅ Converts text into numeric codes — useful before statistical tests or modelling.

#### 🔍 Example 3 — Use `.map()` with a simple function

In [103]:
# Calculate dose group by numeric threshold
df["dose_group"] = df["ledd_mg"].map(lambda x: "High" if x > 600 else "Low")
df[["ledd_mg", "dose_group"]].head()

,ledd_mg,dose_group
0,300,Low
1,500,Low
2,800,High
3,0,Low
4,500,Low


✅ `.map()` passes each value (`x`) into the lambda function and replaces it with the result.

> 🔹 Use this when the transformation depends on one variable.
> 🔹 For multi-column logic, prefer `.apply(axis=1)`.

#### 📊 Example 4 — Numeric mapping to standardised scores


In [104]:
# Example: scale LEDD to g/day
df["ledd_g"] = df["ledd_mg"].map(lambda x: x / 1000 if pd.notna(x) else np.nan)

df[["ledd_mg", "ledd_g"]].head()

,ledd_mg,ledd_g
0,300,0.3
1,500,0.5
2,800,0.8
3,0,0.0
4,500,0.5


| Task                                      | Recommended method     | Example                                                   |
| ----------------------------------------- | ---------------------- | --------------------------------------------------------- |
| Replace or re-label fixed values          | `.map(dict)`           | `df["sex"].map({"Male": "M"})`                            |
| Apply simple 1-to-1 logic                 | `.map(lambda x: ...)`  | `df["ledd_mg"].map(lambda x: "High" if x>600 else "Low")` |
| Apply complex multi-column logic          | `.apply(func, axis=1)` | `df.apply(my_func, axis=1)`                               |
| Apply elementwise across entire DataFrame | `.applymap(func)`      | `df.applymap(lambda x: str(x).lower())`                   |

---


## 🧩 Binning Continuous Data with `pd.cut()`

### 🧠 What is `pd.cut()` and `pd.qcut()`?

`pd.cut()` converts **continuous numeric variables** (like age, BMI, or UPDRS scores) into **discrete categories or “bins”**.

`pd.qcut()` does the same but creates bins with **equal numbers of observations** (quantiles).

This process is called **binning** or **discretisation**.

It’s useful when you want to:

* group continuous data into ranges (e.g., *young*, *middle-aged*, *elderly*);
* count how many patients fall in each range;
* compare distributions visually (bar plots, cross-tabulations);
* prepare data for categorical analysis (e.g., t-tests by severity group).


### 🔢 Example — Bin `age` into age groups

In [105]:
BMI_groups = [0, 18.5, 24.9, 25, 30, 35, 40]
BMI_labels = ["underweight", "normal weight", "overweight", "obesity I", "obesity II", "obesity III"]

df["bmi_group"] = pd.cut(df["BMI"], bins=BMI_groups, labels=BMI_labels, right=True)
df[["patient_id", "BMI", "bmi_group"]].head(8)

,patient_id,BMI,bmi_group
0,P01,22.5,normal weight
1,P02,25.0,overweight
2,P03,30.0,obesity I
3,P04,28.0,obesity I
4,P01,22.7,normal weight
5,P02,24.0,normal weight
6,P03,36.0,obesity III
7,P04,27.6,obesity I


**Explanation:**

* `bins` define numeric intervals (here: 0–39, 40–59, etc.).
* `labels` assign readable names to each range.
* `right=False` means the intervals are *left-closed, right-open* — e.g., 40 ≤ age < 60.

> 💡 You can omit `labels` if you just want the numeric intervals as output.

### 🎚️ Example — Automatic equal-width or quantile bins

#### Equal-width bins:

In [106]:
df["ledd_band"] = pd.cut(df["ledd_mg"], bins=4)
df["ledd_band"].value_counts()

ledd_band
(425.0, 637.5]    4
(-0.85, 212.5]    2
(212.5, 425.0]    2
(637.5, 850.0]    2
Name: count, dtype: int64

Divides LEDD into **4 equal-width intervals** based on its numeric range.

#### Quantile-based bins (equal-sized groups):

In [107]:
df["ledd_quartile"] = pd.qcut(df["ledd_mg"], q=4)
df["ledd_quartile"].value_counts()

ledd_quartile
(-0.001, 325.0]    3
(325.0, 500.0]     3
(587.5, 850.0]     3
(500.0, 587.5]     1
Name: count, dtype: int64

Divides LEDD into **quartiles** (each bin has ~25% of data).
Useful for comparing “lowest-dose” vs “highest-dose” patients.

#### 🧮 Example — Combine `pd.cut()` with `.groupby()`
You can group by the new bins to calculate statistics for each range:

In [108]:
df.groupby("bmi_group")["updrs_iii"].mean().round(1)

/var/folders/nb/mdcnqz1d7673k0zpfm275hrm0000gp/T/ipykernel_65315/2124344277.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df.groupby("bmi_group")["updrs_iii"].mean().round(1)


bmi_group
underweight       NaN
normal weight    26.5
overweight       30.0
obesity I        41.0
obesity II        NaN
obesity III      40.0
Name: updrs_iii, dtype: float64



#### ⚙️ Other useful parameters

| Parameter         | Description                          | Example                         |
| ----------------- | ------------------------------------ | ------------------------------- |
| `bins=`           | Number of bins or list of cut points | `bins=5` or `bins=[0,10,20,30]` |
| `labels=`         | Custom labels for bins               | `labels=["low","mid","high"]`   |
| `right=`          | Include the right edge in intervals  | `right=False` (default is True) |
| `include_lowest=` | Include lowest value in first bin    | `include_lowest=True`           |
| `precision=`      | Decimal precision for bin edges      | `precision=1`                   |
| `duplicates=`     | Handles duplicate bin edges          | `duplicates='drop'`             |

---




## 📚 Unified Summary: pandas Data Transformation, Aggregation & Binning Methods

| Method / Pattern                               | Works on                   | Output shape                | Typical purpose                                                 | When to use                                                                  | Example                                                                                       |
| ---------------------------------------------- | -------------------------- | --------------------------- | --------------------------------------------------------------- | ---------------------------------------------------------------------------- | --------------------------------------------------------------------------------------------- |
| **`.map()`**                                   | Series (one column)        | ✅ Same length               | Simple element-wise transformation using a dictionary or lambda | Quick relabelling or 1-to-1 mapping of values                                | `df["sex"].map({"Male": "M", "Female": "F"})`                                                 |
| **`.map(lambda x: …)`**                        | Series                     | ✅ Same length               | Apply short logic to each element                               | Simple condition using a single column                                       | `df["ledd_mg"].map(lambda x: "High" if x>600 else "Low")`                                     |
| **`.apply(func)`**                             | Series / DataFrame         | ✅ Same length               | Apply a custom function element-wise or row-wise                | When logic depends on multiple columns or more complex rules                 | `df.apply(my_func, axis=1)`                                                                   |
| **`.applymap(func)`**                          | Entire DataFrame           | ✅ Same length               | Apply an element-wise function to every cell                    | Clean or format all values at once                                           | `df.applymap(lambda x: str(x).lower())`                                                       |
| **`.groupby().transform(func)`**               | Grouped Series             | ✅ Same length               | Apply a function *by group* and broadcast result back           | Derive new columns (e.g., per-patient mean, z-scores)                        | `df.groupby("patient_id")["ledd_mg"].transform("mean")`                                       |
| **`.groupby().transform(lambda x: …)`**        | Grouped Series             | ✅ Same length               | Apply custom group-wise logic                                   | Complex per-group operations (e.g., rolling mean, scaling)                   | `df.groupby("patient_id")["ledd_mg"].transform(lambda x: x/x.max())`                          |
| **`.groupby().agg(func)`**                     | Grouped Series / DataFrame | ❌ One row per group         | Compute summary statistics per group                            | Summarise data into a smaller table (per-patient or per-clinic)              | `df.groupby("patient_id")["ledd_mg"].agg(["mean","max","std"])`                               |
| **`.agg(["mean", "max", lambda x: x.std()])`** | Grouped DataFrame          | ❌ One row per group         | Combine built-in and custom stats in one command                | Create descriptive summary reports                                           | `df.groupby("patient_id").agg({"ledd_mg":["mean","std"], "updrs_iii":["mean","count"]})`      |
| **`.transform("mean")`**                       | Grouped Series             | ✅ Same length               | Add group summaries back into original data                     | To attach group averages to every row                                        | `df["mean_ledd_per_patient"] = df.groupby("patient_id")["ledd_mg"].transform("mean")`         |
| **`.apply(lambda x: x.rolling(...).mean())`**  | Series / Grouped Series    | ✅ Same length               | Apply rolling or custom window operations                       | Longitudinal smoothing, time-based metrics                                   | `df.groupby("patient_id")["ledd_mg"].transform(lambda x: x.rolling(365,1).mean())`            |
| **`.agg()` + `reset_index()`**                 | Grouped DataFrame          | ❌ Reduced table             | Store group-level summaries in a separate DataFrame             | When you need a summary table for further analysis or merging                | `patient_summary = df.groupby("patient_id").agg(mean_ledd=("ledd_mg","mean")).reset_index()`  |
| **`pd.cut()`**                                 | Numeric Series             | ✅ Same length (categorical) | Bin continuous variables into fixed numeric ranges              | Create categorical groups from continuous data (e.g., BMI, age, UPDRS bands) | `pd.cut(df["BMI"], [0,18.5,25,30,100], labels=["Underweight","Normal","Overweight","Obese"])` |
| **`pd.qcut()`**                                | Numeric Series             | ✅ Same length (categorical) | Bin continuous variables into quantiles (equal-sized groups)    | Divide continuous data into quartiles, deciles, etc.                         | `pd.qcut(df["ledd_mg"], q=4, labels=["Q1","Q2","Q3","Q4"])`                                   |

---

### 🧭 Quick Guide to Choosing the Right Method

| Goal                                                   | Recommended method                            |
| ------------------------------------------------------ | --------------------------------------------- |
| Replace or recode single-column values                 | `.map(dict)` or `.map(lambda x: …)`           |
| Apply conditional logic involving several columns      | `.apply(func, axis=1)`                        |
| Clean or format *every* cell in a DataFrame            | `.applymap(func)`                             |
| Add derived columns calculated per group (same size)   | `.groupby().transform()`                      |
| Compute summary statistics per group (fewer rows)      | `.groupby().agg()`                            |
| Mix built-in and custom aggregations                   | `.groupby().agg(["mean", lambda x: x.std()])` |
| Compute rolling or windowed metrics                    | `.transform(lambda x: x.rolling(...).mean())` |
| **Convert continuous variables into categorical bins** | **`pd.cut()` or `pd.qcut()`**                 |

---




### 📚 Unified Summary: pandas Data Transformation & Aggregation Methods

| Method / Pattern                               | Works on                   | Output shape        | Typical purpose                                                 | When to use                                                     | Example                                                                                      |
| ---------------------------------------------- | -------------------------- | ------------------- | --------------------------------------------------------------- | --------------------------------------------------------------- | -------------------------------------------------------------------------------------------- |
| **`.map()`**                                   | Series (one column)        | ✅ Same length       | Simple element-wise transformation using a dictionary or lambda | Quick relabelling or 1-to-1 mapping of values                   | `df["sex"].map({"Male": "M", "Female": "F"})`                                                |
| **`.map(lambda x: …)`**                        | Series                     | ✅ Same length       | Apply short logic to each element                               | Simple condition using a single column                          | `df["ledd_mg"].map(lambda x: "High" if x>600 else "Low")`                                    |
| **`.apply(func)`**                             | Series / DataFrame         | ✅ Same length       | Apply a custom function element-wise or row-wise                | When logic depends on multiple columns or more complex rules    | `df.apply(my_func, axis=1)`                                                                  |
| **`.applymap(func)`**                          | Entire DataFrame           | ✅ Same length       | Apply an element-wise function to every cell                    | Clean or format all values at once                              | `df.applymap(lambda x: str(x).lower())`                                                      |
| **`.groupby().transform(func)`**               | Grouped Series             | ✅ Same length       | Apply a function *by group* and broadcast result back           | Derive new columns (e.g., per-patient mean, z-scores)           | `df.groupby("patient_id")["ledd_mg"].transform("mean")`                                      |
| **`.groupby().transform(lambda x: …)`**        | Grouped Series             | ✅ Same length       | Apply custom group-wise logic                                   | Complex per-group operations (e.g., rolling mean, scaling)      | `df.groupby("patient_id")["ledd_mg"].transform(lambda x: x/x.max())`                         |
| **`.groupby().agg(func)`**                     | Grouped Series / DataFrame | ❌ One row per group | Compute summary statistics per group                            | Summarise data into a smaller table (per-patient or per-clinic) | `df.groupby("patient_id")["ledd_mg"].agg(["mean","max","std"])`                              |
| **`.agg(["mean", "max", lambda x: x.std()])`** | Grouped DataFrame          | ❌ One row per group | Combine built-in and custom stats in one command                | Create descriptive summary reports                              | `df.groupby("patient_id").agg({"ledd_mg":["mean","std"], "updrs_iii":["mean","count"]})`     |
| **`.transform("mean")`**                       | Grouped Series             | ✅ Same length       | Add group summaries back into original data                     | To attach group averages to every row                           | `df["mean_ledd_per_patient"] = df.groupby("patient_id")["ledd_mg"].transform("mean")`        |
| **`.apply(lambda x: x.rolling(...).mean())`**  | Series / Grouped Series    | ✅ Same length       | Apply rolling or custom window operations                       | Longitudinal smoothing, time-based metrics                      | `df.groupby("patient_id")["ledd_mg"].transform(lambda x: x.rolling(365,1).mean())`           |
| **`.agg()` + `reset_index()`**                 | Grouped DataFrame          | ❌ Reduced table     | Store group-level summaries in a separate DataFrame             | When you need a summary table for further analysis or merging   | `patient_summary = df.groupby("patient_id").agg(mean_ledd=("ledd_mg","mean")).reset_index()` |



### 🧭 Quick guide to choosing the right method

| Goal                                                 | Recommended method                            |
| ---------------------------------------------------- | --------------------------------------------- |
| Replace or recode single-column values               | `.map(dict)` or `.map(lambda x: …)`           |
| Apply conditional logic involving several columns    | `.apply(func, axis=1)`                        |
| Clean or format *every* cell in a DataFrame          | `.applymap(func)`                             |
| Add derived columns calculated per group (same size) | `.groupby().transform()`                      |
| Compute summary statistics per group (fewer rows)    | `.groupby().agg()`                            |
| Mix built-in and custom aggregations                 | `.groupby().agg(["mean", lambda x: x.std()])` |
| Compute rolling or windowed metrics                  | `.transform(lambda x: x.rolling(...).mean())` |







### 💡 Teaching tip

When deciding between `.map()`, `.apply()`, `.transform()`, and `.agg()`, ask yourself:

1. **Do I want the same number of rows back?**
   → Yes → use `.map()`, `.apply()`, or `.transform()`.
   → No (summary table) → use `.agg()`.

2. **Is my logic simple or multi-column?**
   → One column → `.map()` or `.apply()` on a Series.
   → Several columns → `.apply(axis=1)`.

---



### 🧪 <span style="color:red; font-weight:bold">Your Turn!</span>

1. Use `.map()` to recode `sex` into `"Male" → 1`, `"Female" → 0`.
2. Map `other_medication` ("Yes"/"No") into Boolean `True`/`False`.

4. Check your results using `.value_counts()` for each new column.

---


### 🍎 <span style="color:red; font-weight:bold">Student challenge 

<span style="color:red; font-weight:bold">Now your turn!</span>
>
> 1. Using the dataset `df`, convert the `date_of_birth` and `visit_date` columns to datetime format if they are not already.
> 2. Calculate the patien|t’s **age at last visit**: `visit_date − date_of_birth`.
> 3. Create a new column `updrs_iii_diag` where updrs <32  is 'No PD' and >=32 is 'PD' using .map().
> 4. Create a new category column from pre-cleaned 'medication' column, called `medication_type`, with the following categories:
>    - 'Dopamine Agonist' if medication contains 'rotigotine', 'pramipexole', or 'ropinirole'
>    - 'Levodopa Derivative' if medication contains 'levodopa' or 'carbidopa' or 'co-beneldopa'
>    - 'COMT Inhibitor' if medication contains 'Entacapone' or 'Tolcapone'
>    - 'COMT mixture' if medication contains both 'Levodopa Derivative' and 'COMT Inhibitor'
>    - 'Other' for all other medications
>    - 'None' if no medication is prescribed (empty string)
> 5. Create a column medication_abbreviation with the following mappings:
>    - 'Dopamine Agonist' → 'DA'
>    - 'Levodopa Derivative' → 'LD'
>    - 'COMT Inhibitor' → 'COMT'
>    - 'COMT mixture' → 'COMT+LD'
>    - 'Other' → 'Other'
>    - 'None' → 'None'
> 6. For each category in 'medication_type', use `.groupby()` to calculate the mean and standard deviation of `ledd_mg` and `updrs_iii`.
> 7. Create a new category column `age_group` with bins: '<60', '60-69', '70-79', '80+' based on `age_at_visit`.
> 8. Use `.groupby()` to calculate the mean `updrs_iii` score for each `age_group` and `sex` combination.
> 9. Calculate the rolling window average of ledd over a 365-day period for each patient. Create a new column `ledd_rolling_avg_365` to store this value.
> 10. Compute the mean and SD of `updrs_iii` per patient using `.agg()` and save as `patient_summary`. Merge `patient_summary` back into `df` on `patient_id`.
> 11. Create `updrs_zscore` = (`updrs_iii` − mean) / SD using `.transform()` or `.apply(lambda x: …)`. Count how many visits have `updrs_zscore > 0.5` (above-average motor severity).


In [109]:
df["date_of_birth"]

0    14.05.1950
1    22.08.1965
2    30.11.1970
3    15.01.1980
4    14.05.1950
5    22.08.1965
6    30.11.1970
7    15.01.1980
8    10.10.1960
9    22.05.1980
Name: date_of_birth, dtype: object

In [110]:
df["visit_date"]

0   2023-01-01
1   2023-01-02
2   2023-01-03
3   2023-01-04
4   2024-02-08
5   2025-02-23
6   2023-10-06
7   2024-06-04
8   2024-05-01
9   2025-01-05
Name: visit_date, dtype: datetime64[ns]

In [111]:
# Convert date_of_birth to datetime
df["date_of_birth"] = pd.to_datetime(df["date_of_birth"], format="%d.%m.%Y")
df.dtypes


patient_id                             object
sex                                  category
date_of_birth                  datetime64[ns]
updrs_iii                             float64
ledd_mg                                 int64
visit_date                     datetime64[ns]
medication                             object
other_medication                       object
BMI                                   float64
id_last_update                 datetime64[ns]
ledd_per_updrs                        float64
high_bmi                                int64
ledd_zscore                           float64
is_on_medication                         bool
medication_clean                       object
on_dopa_derivative                       bool
on_dopamine_agonist                      bool
on_combination_therapy                   bool
visit_year                              int32
visit_month                             int32
days_since_update                     float64
months_since_update               

In [112]:
df["age_at_visit"] = (df["visit_date"] - df["date_of_birth"]).dt.days / 365.2
df

,patient_id,sex,date_of_birth,updrs_iii,ledd_mg,visit_date,medication,other_medication,BMI,id_last_update,...,medication_simple_cleaning,ledd_per_bmi,sex_short,other_medication_code,dose_group,ledd_g,bmi_group,ledd_band,ledd_quartile,age_at_visit
0,P01,Male,1950-05-14,22.0,300,2023-01-01,Co-Carbidopa 12.5/50,Yes,22.5,2024-06-04 23:25:00,...,co-carbidopa 12.5/50,13.333333,M,1.0,Low,0.30,normal weight,"(212.5, 425.0]","(-0.001, 325.0]",72.645126
1,P02,Female,1965-08-22,30.0,500,2023-01-02,Rotigotine 0.5,Yes,25.0,2024-10-04 23:25:00,...,rotigotine 0.5,20.000000,F,1.0,Low,0.50,overweight,"(425.0, 637.5]","(325.0, 500.0]",57.371303
2,P03,Female,1970-11-30,40.0,800,2023-01-03,Co-BEneldopa;,No,30.0,2023-09-03 23:25:00,...,co-beneldopa;,26.666667,F,0.0,High,0.80,obesity I,"(637.5, 850.0]","(587.5, 850.0]",52.100219
3,P04,Male,1980-01-15,NaN,0,2023-01-04,,No,28.0,2024-06-04 23:25:00,...,None,0.000000,M,0.0,Low,0.00,obesity I,"(-0.85, 212.5]","(-0.001, 325.0]",42.976451
4,P01,Male,1950-05-14,25.0,500,2024-02-08,"Co-carbidopa,,",Yes,22.7,2024-06-04 23:25:00,...,"co-carbidopa,,",22.026432,M,1.0,Low,0.50,normal weight,"(425.0, 637.5]","(325.0, 500.0]",73.748631
5,P02,Female,1965-08-22,31.0,600,2025-02-23,Rotigotine.,No,24.0,2024-10-04 23:25:00,...,rotigotine.,25.000000,F,0.0,Low,0.60,normal weight,"(425.0, 637.5]","(587.5, 850.0]",59.515334
6,P03,Female,1970-11-30,40.0,850,2023-10-06,LEVODOPA + ENTACAPONE,Yes,36.0,2023-09-03 23:25:00,...,levodopa + entacapone,23.611111,F,1.0,High,0.85,obesity III,"(637.5, 850.0]","(587.5, 850.0]",52.855969
7,P04,Male,1980-01-15,NaN,0,2024-06-04,N/A,No,27.6,2024-06-04 23:25:00,...,n/a,0.000000,M,0.0,Low,0.00,obesity I,"(-0.85, 212.5]","(-0.001, 325.0]",44.392114
8,P05,Male,1960-10-10,42.0,550,2024-05-01,Entacapone 200,NaN,26.4,NaT,...,entacapone 200,20.833333,M,NaN,Low,0.55,obesity I,"(425.0, 637.5]","(500.0, 587.5]",63.565170
9,P06,Unidentified,1980-05-22,28.0,400,2025-01-05,Levodopa,No,23.4,2025-06-04 23:25:00,...,levodopa,17.094017,NaN,0.0,Low,0.40,normal weight,"(212.5, 425.0]","(325.0, 500.0]",44.630340


In [113]:
# Create a updrs_iii_diag column where updrs <32  is 'No PD' and >=32 is 'PD'
df["updrs_iii_diag"] = df["updrs_iii"].map(lambda x: "No PD" if pd.notna(x) and x < 32 else ("PD" if pd.notna(x) and x >= 32 else np.nan))
df[["updrs_iii", "updrs_iii_diag"]]

,updrs_iii,updrs_iii_diag
0,22.0,No PD
1,30.0,No PD
2,40.0,PD
3,NaN,NaN
4,25.0,No PD
5,31.0,No PD
6,40.0,PD
7,NaN,NaN
8,42.0,PD
9,28.0,No PD


In [114]:
# Create a column called 'medication_type' based on keywords in 'medication_clean'
def classify_medication(med):   
    if pd.isna(med) or med == "":
        return "None"
    med = med.lower()

    if any(x in med for x in ["rotigotine", "pramipexole", "ropinirole"]):
        return "Dopamine Agonist"
    elif ("levodopa" in med or "carbidopa" in med) and ("entacapone" in med or "tolcapone" in med):
        return "COMT mixture"
    elif any(x in med for x in ["levodopa", "carbidopa", "co-beneldopa"]):
        return "Levodopa Derivative"
    elif any(x in med for x in ["entacapone", "tolcapone"]):
        return "COMT Inhibitor"
    else:
        return "Other"

# Apply it to the cleaned medication column
df["medication_type"] = df["medication_clean"].apply(classify_medication)

# Convert to categorical type for efficiency
df["medication_type"] = df["medication_type"].astype("category")

df[["medication_clean", "medication_type"]]


,medication_clean,medication_type
0,co-carbidopa,Levodopa Derivative
1,rotigotine,Dopamine Agonist
2,co-beneldopa,Levodopa Derivative
3,,None
4,co-carbidopa,Levodopa Derivative
5,rotigotine,Dopamine Agonist
6,levodopa + entacapone,COMT mixture
7,,None
8,entacapone,COMT Inhibitor
9,levodopa,Levodopa Derivative


In [115]:
# create an abbreviated medication type column
abbrev_map = {
    "Dopamine Agonist": "DA",
    "Levodopa Derivative": "LD",
    "COMT mixture": "COMT+LDR",
    "COMT Inhibitor": "COMT",
    "Other": "Other",
    "None": "None"
}
df["medication_type_abbrev"] = df["medication_type"].map(abbrev_map)
df[["medication_type", "medication_type_abbrev"]]

,medication_type,medication_type_abbrev
0,Levodopa Derivative,LD
1,Dopamine Agonist,DA
2,Levodopa Derivative,LD
3,None,None
4,Levodopa Derivative,LD
5,Dopamine Agonist,DA
6,COMT mixture,COMT+LDR
7,None,None
8,COMT Inhibitor,COMT
9,Levodopa Derivative,LD


In [116]:
# For each category of medication_type, calculate the mean and SD of ledd_mg using .groupby()
df.groupby("medication_type")["ledd_mg"].agg(["mean", "std", "count"])

/var/folders/nb/mdcnqz1d7673k0zpfm275hrm0000gp/T/ipykernel_65315/3912852306.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df.groupby("medication_type")["ledd_mg"].agg(["mean", "std", "count"])


,mean,std,count
medication_type,,,
COMT Inhibitor,550.0,NaN,1
COMT mixture,850.0,NaN,1
Dopamine Agonist,550.0,70.710678,2
Levodopa Derivative,500.0,216.024690,4
None,0.0,0.000000,2


In [117]:
#  Create a new category column `age_group` with bins: '<60', '60-69', '70-79', '80+' based on `age_at_visit`.

bins = [0, 59, 69, 79, np.inf]
labels = ['<60', '60-69', '70-79', '80+']
df["age_group"] = pd.cut(df["age_at_visit"], bins=bins, labels=labels   , right=True)
df[["age_at_visit", "age_group"]]

,age_at_visit,age_group
0,72.645126,70-79
1,57.371303,<60
2,52.100219,<60
3,42.976451,<60
4,73.748631,70-79
5,59.515334,60-69
6,52.855969,<60
7,44.392114,<60
8,63.565170,60-69
9,44.630340,<60


In [118]:

# Use `.groupby()` to calculate the mean `updrs_iii` score for each `age_group` and `sex` combination.
df.groupby(["age_group", "sex"])["updrs_iii"].mean()


/var/folders/nb/mdcnqz1d7673k0zpfm275hrm0000gp/T/ipykernel_65315/1849921380.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df.groupby(["age_group", "sex"])["updrs_iii"].mean()


age_group  sex         
<60        Female          36.666667
           Male                  NaN
           Unidentified    28.000000
60-69      Female          31.000000
           Male            42.000000
           Unidentified          NaN
70-79      Female                NaN
           Male            23.500000
           Unidentified          NaN
80+        Female                NaN
           Male                  NaN
           Unidentified          NaN
Name: updrs_iii, dtype: float64

In [120]:
 df

,patient_id,sex,date_of_birth,updrs_iii,ledd_mg,visit_date,medication,other_medication,BMI,id_last_update,...,dose_group,ledd_g,bmi_group,ledd_band,ledd_quartile,age_at_visit,updrs_iii_diag,medication_type,medication_type_abbrev,age_group
0,P01,Male,1950-05-14,22.0,300,2023-01-01,Co-Carbidopa 12.5/50,Yes,22.5,2024-06-04 23:25:00,...,Low,0.30,normal weight,"(212.5, 425.0]","(-0.001, 325.0]",72.645126,No PD,Levodopa Derivative,LD,70-79
4,P01,Male,1950-05-14,25.0,500,2024-02-08,"Co-carbidopa,,",Yes,22.7,2024-06-04 23:25:00,...,Low,0.50,normal weight,"(425.0, 637.5]","(325.0, 500.0]",73.748631,No PD,Levodopa Derivative,LD,70-79
1,P02,Female,1965-08-22,30.0,500,2023-01-02,Rotigotine 0.5,Yes,25.0,2024-10-04 23:25:00,...,Low,0.50,overweight,"(425.0, 637.5]","(325.0, 500.0]",57.371303,No PD,Dopamine Agonist,DA,<60
5,P02,Female,1965-08-22,31.0,600,2025-02-23,Rotigotine.,No,24.0,2024-10-04 23:25:00,...,Low,0.60,normal weight,"(425.0, 637.5]","(587.5, 850.0]",59.515334,No PD,Dopamine Agonist,DA,60-69
2,P03,Female,1970-11-30,40.0,800,2023-01-03,Co-BEneldopa;,No,30.0,2023-09-03 23:25:00,...,High,0.80,obesity I,"(637.5, 850.0]","(587.5, 850.0]",52.100219,PD,Levodopa Derivative,LD,<60
6,P03,Female,1970-11-30,40.0,850,2023-10-06,LEVODOPA + ENTACAPONE,Yes,36.0,2023-09-03 23:25:00,...,High,0.85,obesity III,"(637.5, 850.0]","(587.5, 850.0]",52.855969,PD,COMT mixture,COMT+LDR,<60
3,P04,Male,1980-01-15,NaN,0,2023-01-04,,No,28.0,2024-06-04 23:25:00,...,Low,0.00,obesity I,"(-0.85, 212.5]","(-0.001, 325.0]",42.976451,NaN,None,None,<60
7,P04,Male,1980-01-15,NaN,0,2024-06-04,N/A,No,27.6,2024-06-04 23:25:00,...,Low,0.00,obesity I,"(-0.85, 212.5]","(-0.001, 325.0]",44.392114,NaN,None,None,<60
8,P05,Male,1960-10-10,42.0,550,2024-05-01,Entacapone 200,NaN,26.4,NaT,...,Low,0.55,obesity I,"(425.0, 637.5]","(500.0, 587.5]",63.565170,PD,COMT Inhibitor,COMT,60-69
9,P06,Unidentified,1980-05-22,28.0,400,2025-01-05,Levodopa,No,23.4,2025-06-04 23:25:00,...,Low,0.40,normal weight,"(212.5, 425.0]","(325.0, 500.0]",44.630340,No PD,Levodopa Derivative,LD,<60


In [131]:
df[df["visit_date"].isna()] 

,patient_id,sex,date_of_birth,updrs_iii,ledd_mg,visit_date,medication,other_medication,BMI,id_last_update,...,ledd_g,bmi_group,ledd_band,ledd_quartile,age_at_visit,updrs_iii_diag,medication_type,medication_type_abbrev,age_group,ledd_rolling_avg_365


In [132]:
df[df["ledd_mg"].isna()]  

,patient_id,sex,date_of_birth,updrs_iii,ledd_mg,visit_date,medication,other_medication,BMI,id_last_update,...,ledd_g,bmi_group,ledd_band,ledd_quartile,age_at_visit,updrs_iii_diag,medication_type,medication_type_abbrev,age_group,ledd_rolling_avg_365


In [141]:
df.groupby("patient_id").rolling("365D", on="visit_date")["ledd_mg"].mean()


/var/folders/nb/mdcnqz1d7673k0zpfm275hrm0000gp/T/ipykernel_65315/15656578.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df.groupby("patient_id").rolling("365D", on="visit_date")["ledd_mg"].mean()


patient_id  visit_date
P01         2023-01-01    300.0
            2024-02-08    500.0
P02         2023-01-02    500.0
            2025-02-23    600.0
P03         2023-01-03    800.0
            2023-10-06    825.0
P04         2023-01-04      0.0
            2024-06-04      0.0
P05         2024-05-01    550.0
P06         2025-01-05    400.0
Name: ledd_mg, dtype: float64

In [138]:
df.drop("ledd_rolling_avg_365", axis=1, inplace=True)

In [143]:
rolled

visit_date
2023-01-01    300.0
2024-02-08    500.0
2023-01-02    500.0
2025-02-23    600.0
2023-01-03    800.0
2023-10-06    825.0
2023-01-04      0.0
2024-06-04      0.0
2024-05-01    550.0
2025-01-05    400.0
Name: ledd_mg, dtype: float64

In [ ]:
# Calculate a rolling window average of ledd over a 365-day period for each patient

# Ensure visit_date is datetime
# Coerce types (this is the usual culprit for all-NaN results)
df["visit_date"] = pd.to_datetime(df["visit_date"], errors="coerce")
df["ledd_mg"]    = pd.to_numeric(df["ledd_mg"], errors="coerce")


# Convert patient_id to category if not already (optional)
df["patient_id"] = df["patient_id"].astype("category")

# Sort by date within patient
df = df.sort_values(["patient_id", "visit_date"])

# Let's see how the rolling mean looks like
display( df.groupby("patient_id").rolling("365D", on="visit_date")["ledd_mg"].mean())

# 365-day rolling average of ledd_mg within each patient
rolled = (
    df.groupby("patient_id")
      .rolling("365D", on="visit_date")["ledd_mg"]
      .mean()
      .reset_index(level=0, drop=True)   # <- drop patient_id level
)

display(rolled)

df["ledd_rolling_avg_365"] = rolled.reset_index(drop=True) # <- drop the indexing by the the visit  


# Inspect
df[["patient_id", "visit_date", "ledd_mg", "ledd_rolling_avg_365"]]


/var/folders/nb/mdcnqz1d7673k0zpfm275hrm0000gp/T/ipykernel_65315/4070061924.py:16: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  display( df.groupby("patient_id").rolling("365D", on="visit_date")["ledd_mg"].mean())


patient_id  visit_date
P01         2023-01-01    300.0
            2024-02-08    500.0
P02         2023-01-02    500.0
            2025-02-23    600.0
P03         2023-01-03    800.0
            2023-10-06    825.0
P04         2023-01-04      0.0
            2024-06-04      0.0
P05         2024-05-01    550.0
P06         2025-01-05    400.0
Name: ledd_mg, dtype: float64

/var/folders/nb/mdcnqz1d7673k0zpfm275hrm0000gp/T/ipykernel_65315/4070061924.py:20: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df.groupby("patient_id")


visit_date
2023-01-01    300.0
2024-02-08    500.0
2023-01-02    500.0
2025-02-23    600.0
2023-01-03    800.0
2023-10-06    825.0
2023-01-04      0.0
2024-06-04      0.0
2024-05-01    550.0
2025-01-05    400.0
Name: ledd_mg, dtype: float64

,patient_id,visit_date,ledd_mg,ledd_rolling_avg_365
0,P01,2023-01-01,300,NaN
4,P01,2024-02-08,500,NaN
1,P02,2023-01-02,500,NaN
5,P02,2025-02-23,600,NaN
2,P03,2023-01-03,800,NaN
6,P03,2023-10-06,850,NaN
3,P04,2023-01-04,0,NaN
7,P04,2024-06-04,0,NaN
8,P05,2024-05-01,550,NaN
9,P06,2025-01-05,400,NaN


In [ ]:
#10. Compute the mean and SD of `updrs_iii` per patient using `.agg()` and save as `patient_summary`. Merge `patient_summary` back into `df` on `patient_id`.
patient_summary = (
    df.groupby("patient_id")
      .agg({
          "updrs_iii": ["mean", "std", "min", "max", "count"]
      })
)
patient_summary.columns = ['_'.join(col).strip() for col in patient_summary.columns.values]
patient_summary.reset_index(inplace=True)
df_new = df.merge(patient_summary, on="patient_id", how="left")
df_new.head()

,patient_id,sex,date_of_birth,updrs_iii,ledd_mg,visit_date,medication,other_medication,BMI,id_last_update,...,bmi_group,ledd_band,ledd_quartile,medication_type_abbrev,updrs_iii_diag,updrs_iii_mean,updrs_iii_std,updrs_iii_min,updrs_iii_max,updrs_iii_count
0,P01,Male,1950-05-14,22.0,300,2023-01-01,Co-Carbidopa 12.5/50,Yes,22.5,2024-06-04 23:25:00,...,normal weight,"(212.5, 425.0]","(-0.001, 325.0]",LD,No PD,23.5,2.121320,22.0,25.0,2
1,P02,Female,1965-08-22,30.0,500,2023-01-02,Rotigotine 0.5,Yes,25.0,2024-10-04 23:25:00,...,overweight,"(425.0, 637.5]","(325.0, 500.0]",DA,No PD,30.5,0.707107,30.0,31.0,2
2,P03,Female,1970-11-30,40.0,800,2023-01-03,Co-BEneldopa;,No,30.0,2023-09-03 23:25:00,...,obesity I,"(637.5, 850.0]","(587.5, 850.0]",LD,PD,40.0,0.000000,40.0,40.0,2
3,P04,Male,1980-01-15,NaN,0,2023-01-04,,No,28.0,2024-06-04 23:25:00,...,obesity I,"(-0.85, 212.5]","(-0.001, 325.0]",None,NaN,NaN,NaN,NaN,NaN,0
4,P01,Male,1950-05-14,25.0,500,2024-02-08,"Co-carbidopa,,",Yes,22.7,2024-06-04 23:25:00,...,normal weight,"(425.0, 637.5]","(325.0, 500.0]",LD,No PD,23.5,2.121320,22.0,25.0,2


In [ ]:
# Create `updrs_zscore` = (`updrs_iii` − mean) / SD using `.transform()` or `.apply(lambda x: …)`. Count how many visits have `updrs_zscore > 1` (above-average motor severity).
df["updrs_zscore"] = (
    df.groupby("patient_id")["updrs_iii"]
      .transform(lambda x: (x - x.mean()) / x.std())
)
df[["patient_id", "updrs_iii", "updrs_zscore"]]



,patient_id,updrs_iii,updrs_zscore
0,P01,22.0,-0.707107
1,P02,30.0,-0.707107
2,P03,40.0,NaN
3,P04,NaN,NaN
4,P01,25.0,0.707107
5,P02,31.0,0.707107
6,P03,40.0,NaN
7,P04,NaN,NaN
8,P05,42.0,NaN
9,P06,28.0,NaN


In [ ]:
#count rows with z-score > 0.5

print('Number of entries with above average UPDRS score:', df[df["updrs_zscore"] > 0.5].shape[0])


Number of entries with above average UPDRS score: 2



## 🔗 Merging / Joining Datasets in pandas

In real-world healthcare analytics, data often lives in **multiple tables**:

* 🧍‍♀️ **Patients table** — demographics, IDs, scores
* 🏥 **Clinics table** — clinic location, region, deprivation index
* 💊 **Medication or visits table** — prescriptions, doses, visit dates

To analyse the data together, you must **merge** (or “join”) these tables into one.

pandas’ `merge()` works similarly to **SQL joins**, letting you combine datasets based on shared columns (keys).

### ⚙️ Syntax
```python
pd.merge(left, right, on="key_column", how="join_type")
```

| Parameter       | Description                                                |
| --------------- | ---------------------------------------------------------- |
| `left`, `right` | DataFrames you want to merge                               |
| `on`            | Column(s) they share in common (the “key”)                 |
| `how`           | Type of join: `'inner'`, `'left'`, `'right'`, or `'outer'` |



---
### 🧠 Types of merges and when to use them

| Join type   | Keeps rows from…                                                   | Description                                                                    | When to use (clinical example)                                                                             |
| ----------- | ------------------------------------------------------------------ | ------------------------------------------------------------------------------ | ---------------------------------------------------------------------------------------------------------- |
| **`inner`** | Only rows with matching keys in *both* tables                      | Keeps **intersection** — only records present in both DataFrames               | When you only want patients who appear in both datasets (e.g., only those with both visit and clinic info) |
| **`left`**  | All rows from the **left** DataFrame, matching info from the right | Keeps **everything from your main dataset**; fills unmatched values with `NaN` | Most common: when patient-level data is primary and you want to add clinic info (even if some don’t match) |
| **`right`** | All rows from the **right** DataFrame, matching info from the left | Mirror of left join — keeps all right rows                                     | When clinic data is primary and you only care about clinics even if some have no patients                  |
| **`outer`** | All rows from **both** DataFrames                                  | Keeps **union** — everything, matched or not                                   | When you want a full overview of all patients and all clinics, even if some don’t overlap                  |
| **`cross`** | All combinations (cartesian product)                               | Every row in left joins with every row in right                                | Rarely used in EHR data; useful for simulation or all-pairs comparisons                                    |

---


### 🔍 Visual summary (conceptual)

Imagine merging two datasets by `patient_id`:

```
df (patients):          extra_info (extra)
P01  ✓                  P01  ✓
P02  ✓                  P02  ✓
P03  ✓                  P05  ✓
P04  ✓                  
```

| Join type | Result                                               |
| --------- | ---------------------------------------------------- |
| `inner`   | P01, P02, P03  → Only matches present in both        |
| `left`    | P01, P02, P03, P04 → Keeps all from df, adds matches |
| `right`   | P01, P02, P03, P05 → Keeps all from extra_info       |
| `outer`   | P01, P02, P03, P04, P05 → Everything                 |

![](images/IMG_2880.JPG)

### 🧩 Example: Two supplementary tables

In [ ]:
# Table 1: additional patient info
extra_info = pd.DataFrame({
    "patient_id": ["P01", "P02", "P03", "P04", "P07"],
    "height_cm": [172, 160, 158, 182 , 175],
    "weight_kg": [72, 65, 60, 80, 70],
    "smoking_status": ["Former", "Never", "Current", "Never", "Former"],
    "clinic_id": [101, 102, 101, 103, 102],
})

# Table 2: clinic information
clinics = pd.DataFrame({
    "clinic_id": [101, 102, 103],
    "clinic_name": ["St. Mary’s", "Queen Elizabeth", "Royal Infirmary"],
    "deprivation_index": [0.25, 0.52, 0.41],
    "region": ["London", "Birmingham", "Manchester"]
})


### 🔗 Merge examples
---

#### 1️⃣ Left join (most common)

Keep all patient rows; add extra info where available.

In [ ]:
df_merged = df.merge(extra_info, on="patient_id", how="left")
df_merged 

,patient_id,sex,date_of_birth,updrs_iii,ledd_mg,visit_date,medication,other_medication,BMI,id_last_update,...,bmi_group,ledd_band,ledd_quartile,medication_type_abbrev,updrs_iii_diag,updrs_zscore,height_cm,weight_kg,smoking_status,clinic_id
0,P01,Male,1950-05-14,22.0,300,2023-01-01,Co-Carbidopa 12.5/50,Yes,22.5,2024-06-04 23:25:00,...,normal weight,"(212.5, 425.0]","(-0.001, 325.0]",LD,No PD,-0.707107,172.0,72.0,Former,101.0
1,P02,Female,1965-08-22,30.0,500,2023-01-02,Rotigotine 0.5,Yes,25.0,2024-10-04 23:25:00,...,overweight,"(425.0, 637.5]","(325.0, 500.0]",DA,No PD,-0.707107,160.0,65.0,Never,102.0
2,P03,Female,1970-11-30,40.0,800,2023-01-03,Co-BEneldopa;,No,30.0,2023-09-03 23:25:00,...,obesity I,"(637.5, 850.0]","(587.5, 850.0]",LD,PD,NaN,158.0,60.0,Current,101.0
3,P04,Male,1980-01-15,NaN,0,2023-01-04,,No,28.0,2024-06-04 23:25:00,...,obesity I,"(-0.85, 212.5]","(-0.001, 325.0]",None,NaN,NaN,182.0,80.0,Never,103.0
4,P01,Male,1950-05-14,25.0,500,2024-02-08,"Co-carbidopa,,",Yes,22.7,2024-06-04 23:25:00,...,normal weight,"(425.0, 637.5]","(325.0, 500.0]",LD,No PD,0.707107,172.0,72.0,Former,101.0
5,P02,Female,1965-08-22,31.0,600,2025-02-23,Rotigotine.,No,24.0,2024-10-04 23:25:00,...,normal weight,"(425.0, 637.5]","(587.5, 850.0]",DA,No PD,0.707107,160.0,65.0,Never,102.0
6,P03,Female,1970-11-30,40.0,850,2023-10-06,LEVODOPA + ENTACAPONE,Yes,36.0,2023-09-03 23:25:00,...,obesity III,"(637.5, 850.0]","(587.5, 850.0]",COMT+LDR,PD,NaN,158.0,60.0,Current,101.0
7,P04,Male,1980-01-15,NaN,0,2024-06-04,N/A,No,27.6,2024-06-04 23:25:00,...,obesity I,"(-0.85, 212.5]","(-0.001, 325.0]",None,NaN,NaN,182.0,80.0,Never,103.0
8,P05,Male,1960-10-10,42.0,550,2024-05-01,Entacapone 200,NaN,26.4,NaT,...,obesity I,"(425.0, 637.5]","(500.0, 587.5]",COMT,PD,NaN,NaN,NaN,NaN,NaN
9,P06,Unidentified,1980-05-22,28.0,400,2025-01-05,Levodopa,No,23.4,2025-06-04 23:25:00,...,normal weight,"(212.5, 425.0]","(325.0, 500.0]",LD,No PD,NaN,NaN,NaN,NaN,NaN


In [ ]:
df_merged["patient_id"].unique()

array(['P01', 'P02', 'P03', 'P04', 'P05', 'P06'], dtype=object)

✅ Keeps all patients, even if some have no matching `extra_info`.
Unmatched fields become `NaN`.

#### 2️⃣ Inner join

Keep only patients who exist in *both* `df` and `extra_info`.

In [ ]:
df_inner = df.merge(extra_info, on="patient_id", how="inner")
df_inner

,patient_id,sex,date_of_birth,updrs_iii,ledd_mg,visit_date,medication,other_medication,BMI,id_last_update,...,bmi_group,ledd_band,ledd_quartile,medication_type_abbrev,updrs_iii_diag,updrs_zscore,height_cm,weight_kg,smoking_status,clinic_id
0,P01,Male,1950-05-14,22.0,300,2023-01-01,Co-Carbidopa 12.5/50,Yes,22.5,2024-06-04 23:25:00,...,normal weight,"(212.5, 425.0]","(-0.001, 325.0]",LD,No PD,-0.707107,172,72,Former,101
1,P02,Female,1965-08-22,30.0,500,2023-01-02,Rotigotine 0.5,Yes,25.0,2024-10-04 23:25:00,...,overweight,"(425.0, 637.5]","(325.0, 500.0]",DA,No PD,-0.707107,160,65,Never,102
2,P03,Female,1970-11-30,40.0,800,2023-01-03,Co-BEneldopa;,No,30.0,2023-09-03 23:25:00,...,obesity I,"(637.5, 850.0]","(587.5, 850.0]",LD,PD,NaN,158,60,Current,101
3,P04,Male,1980-01-15,NaN,0,2023-01-04,,No,28.0,2024-06-04 23:25:00,...,obesity I,"(-0.85, 212.5]","(-0.001, 325.0]",None,NaN,NaN,182,80,Never,103
4,P01,Male,1950-05-14,25.0,500,2024-02-08,"Co-carbidopa,,",Yes,22.7,2024-06-04 23:25:00,...,normal weight,"(425.0, 637.5]","(325.0, 500.0]",LD,No PD,0.707107,172,72,Former,101
5,P02,Female,1965-08-22,31.0,600,2025-02-23,Rotigotine.,No,24.0,2024-10-04 23:25:00,...,normal weight,"(425.0, 637.5]","(587.5, 850.0]",DA,No PD,0.707107,160,65,Never,102
6,P03,Female,1970-11-30,40.0,850,2023-10-06,LEVODOPA + ENTACAPONE,Yes,36.0,2023-09-03 23:25:00,...,obesity III,"(637.5, 850.0]","(587.5, 850.0]",COMT+LDR,PD,NaN,158,60,Current,101
7,P04,Male,1980-01-15,NaN,0,2024-06-04,N/A,No,27.6,2024-06-04 23:25:00,...,obesity I,"(-0.85, 212.5]","(-0.001, 325.0]",None,NaN,NaN,182,80,Never,103


In [ ]:
df_inner["patient_id"].unique()

array(['P01', 'P02', 'P03', 'P04'], dtype=object)



✅ Removes any patients not found in both — useful for analysis requiring complete records.

---
#### 3️⃣ Right join

Keep all rows from `extra_info`, even if some patients aren’t in `df`.

In [ ]:
df_right = df.merge(extra_info, on="patient_id", how="right")
df_right

,patient_id,sex,date_of_birth,updrs_iii,ledd_mg,visit_date,medication,other_medication,BMI,id_last_update,...,bmi_group,ledd_band,ledd_quartile,medication_type_abbrev,updrs_iii_diag,updrs_zscore,height_cm,weight_kg,smoking_status,clinic_id
0,P01,Male,1950-05-14,22.0,300.0,2023-01-01,Co-Carbidopa 12.5/50,Yes,22.5,2024-06-04 23:25:00,...,normal weight,"(212.5, 425.0]","(-0.001, 325.0]",LD,No PD,-0.707107,172,72,Former,101
1,P01,Male,1950-05-14,25.0,500.0,2024-02-08,"Co-carbidopa,,",Yes,22.7,2024-06-04 23:25:00,...,normal weight,"(425.0, 637.5]","(325.0, 500.0]",LD,No PD,0.707107,172,72,Former,101
2,P02,Female,1965-08-22,30.0,500.0,2023-01-02,Rotigotine 0.5,Yes,25.0,2024-10-04 23:25:00,...,overweight,"(425.0, 637.5]","(325.0, 500.0]",DA,No PD,-0.707107,160,65,Never,102
3,P02,Female,1965-08-22,31.0,600.0,2025-02-23,Rotigotine.,No,24.0,2024-10-04 23:25:00,...,normal weight,"(425.0, 637.5]","(587.5, 850.0]",DA,No PD,0.707107,160,65,Never,102
4,P03,Female,1970-11-30,40.0,800.0,2023-01-03,Co-BEneldopa;,No,30.0,2023-09-03 23:25:00,...,obesity I,"(637.5, 850.0]","(587.5, 850.0]",LD,PD,NaN,158,60,Current,101
5,P03,Female,1970-11-30,40.0,850.0,2023-10-06,LEVODOPA + ENTACAPONE,Yes,36.0,2023-09-03 23:25:00,...,obesity III,"(637.5, 850.0]","(587.5, 850.0]",COMT+LDR,PD,NaN,158,60,Current,101
6,P04,Male,1980-01-15,NaN,0.0,2023-01-04,,No,28.0,2024-06-04 23:25:00,...,obesity I,"(-0.85, 212.5]","(-0.001, 325.0]",None,NaN,NaN,182,80,Never,103
7,P04,Male,1980-01-15,NaN,0.0,2024-06-04,N/A,No,27.6,2024-06-04 23:25:00,...,obesity I,"(-0.85, 212.5]","(-0.001, 325.0]",None,NaN,NaN,182,80,Never,103
8,P07,NaN,NaT,NaN,NaN,NaT,NaN,NaN,NaN,NaT,...,NaN,NaN,NaN,NaN,NaN,NaN,175,70,Former,102


In [ ]:
df_right["patient_id"].unique()

array(['P01', 'P02', 'P03', 'P04', 'P07'], dtype=object)


✅ Useful when you want to check which patients in a registry are missing from your primary dataset.

---

#### 4️⃣ Outer join

Keep *everything* from both sides.

In [ ]:
df_outer = df.merge(extra_info, on="patient_id", how="outer")
df_outer

,patient_id,sex,date_of_birth,updrs_iii,ledd_mg,visit_date,medication,other_medication,BMI,id_last_update,...,bmi_group,ledd_band,ledd_quartile,medication_type_abbrev,updrs_iii_diag,updrs_zscore,height_cm,weight_kg,smoking_status,clinic_id
0,P01,Male,1950-05-14,22.0,300.0,2023-01-01,Co-Carbidopa 12.5/50,Yes,22.5,2024-06-04 23:25:00,...,normal weight,"(212.5, 425.0]","(-0.001, 325.0]",LD,No PD,-0.707107,172.0,72.0,Former,101.0
1,P01,Male,1950-05-14,25.0,500.0,2024-02-08,"Co-carbidopa,,",Yes,22.7,2024-06-04 23:25:00,...,normal weight,"(425.0, 637.5]","(325.0, 500.0]",LD,No PD,0.707107,172.0,72.0,Former,101.0
2,P02,Female,1965-08-22,30.0,500.0,2023-01-02,Rotigotine 0.5,Yes,25.0,2024-10-04 23:25:00,...,overweight,"(425.0, 637.5]","(325.0, 500.0]",DA,No PD,-0.707107,160.0,65.0,Never,102.0
3,P02,Female,1965-08-22,31.0,600.0,2025-02-23,Rotigotine.,No,24.0,2024-10-04 23:25:00,...,normal weight,"(425.0, 637.5]","(587.5, 850.0]",DA,No PD,0.707107,160.0,65.0,Never,102.0
4,P03,Female,1970-11-30,40.0,800.0,2023-01-03,Co-BEneldopa;,No,30.0,2023-09-03 23:25:00,...,obesity I,"(637.5, 850.0]","(587.5, 850.0]",LD,PD,NaN,158.0,60.0,Current,101.0
5,P03,Female,1970-11-30,40.0,850.0,2023-10-06,LEVODOPA + ENTACAPONE,Yes,36.0,2023-09-03 23:25:00,...,obesity III,"(637.5, 850.0]","(587.5, 850.0]",COMT+LDR,PD,NaN,158.0,60.0,Current,101.0
6,P04,Male,1980-01-15,NaN,0.0,2023-01-04,,No,28.0,2024-06-04 23:25:00,...,obesity I,"(-0.85, 212.5]","(-0.001, 325.0]",None,NaN,NaN,182.0,80.0,Never,103.0
7,P04,Male,1980-01-15,NaN,0.0,2024-06-04,N/A,No,27.6,2024-06-04 23:25:00,...,obesity I,"(-0.85, 212.5]","(-0.001, 325.0]",None,NaN,NaN,182.0,80.0,Never,103.0
8,P05,Male,1960-10-10,42.0,550.0,2024-05-01,Entacapone 200,NaN,26.4,NaT,...,obesity I,"(425.0, 637.5]","(500.0, 587.5]",COMT,PD,NaN,NaN,NaN,NaN,NaN
9,P06,Unidentified,1980-05-22,28.0,400.0,2025-01-05,Levodopa,No,23.4,2025-06-04 23:25:00,...,normal weight,"(212.5, 425.0]","(325.0, 500.0]",LD,No PD,NaN,NaN,NaN,NaN,NaN


In [ ]:
df_outer["patient_id"].unique()

array(['P01', 'P02', 'P03', 'P04', 'P05', 'P06', 'P07'], dtype=object)


✅ Keeps all patients — fills missing columns with `NaN`.
Great for **data audits** to find mismatched IDs.

---

#### 5️⃣ Merge on a different key

Once you’ve added `clinic_id` to patients, you can merge with the clinic table:

In [ ]:
df_merged = df_merged.merge(clinics, on="clinic_id", how="left")
df_merged

,patient_id,sex,date_of_birth,updrs_iii,ledd_mg,visit_date,medication,other_medication,BMI,id_last_update,...,medication_type_abbrev,updrs_iii_diag,updrs_zscore,height_cm,weight_kg,smoking_status,clinic_id,clinic_name,deprivation_index,region
0,P01,Male,1950-05-14,22.0,300,2023-01-01,Co-Carbidopa 12.5/50,Yes,22.5,2024-06-04 23:25:00,...,LD,No PD,-0.707107,172.0,72.0,Former,101.0,St. Mary’s,0.25,London
1,P02,Female,1965-08-22,30.0,500,2023-01-02,Rotigotine 0.5,Yes,25.0,2024-10-04 23:25:00,...,DA,No PD,-0.707107,160.0,65.0,Never,102.0,Queen Elizabeth,0.52,Birmingham
2,P03,Female,1970-11-30,40.0,800,2023-01-03,Co-BEneldopa;,No,30.0,2023-09-03 23:25:00,...,LD,PD,NaN,158.0,60.0,Current,101.0,St. Mary’s,0.25,London
3,P04,Male,1980-01-15,NaN,0,2023-01-04,,No,28.0,2024-06-04 23:25:00,...,None,NaN,NaN,182.0,80.0,Never,103.0,Royal Infirmary,0.41,Manchester
4,P01,Male,1950-05-14,25.0,500,2024-02-08,"Co-carbidopa,,",Yes,22.7,2024-06-04 23:25:00,...,LD,No PD,0.707107,172.0,72.0,Former,101.0,St. Mary’s,0.25,London
5,P02,Female,1965-08-22,31.0,600,2025-02-23,Rotigotine.,No,24.0,2024-10-04 23:25:00,...,DA,No PD,0.707107,160.0,65.0,Never,102.0,Queen Elizabeth,0.52,Birmingham
6,P03,Female,1970-11-30,40.0,850,2023-10-06,LEVODOPA + ENTACAPONE,Yes,36.0,2023-09-03 23:25:00,...,COMT+LDR,PD,NaN,158.0,60.0,Current,101.0,St. Mary’s,0.25,London
7,P04,Male,1980-01-15,NaN,0,2024-06-04,N/A,No,27.6,2024-06-04 23:25:00,...,None,NaN,NaN,182.0,80.0,Never,103.0,Royal Infirmary,0.41,Manchester
8,P05,Male,1960-10-10,42.0,550,2024-05-01,Entacapone 200,NaN,26.4,NaT,...,COMT,PD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,P06,Unidentified,1980-05-22,28.0,400,2025-01-05,Levodopa,No,23.4,2025-06-04 23:25:00,...,LD,No PD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN



Now you can analyse LEDD or UPDRS by **clinic region** or **deprivation index**.

### 🧠 When to use which

| Goal                                              | Recommended join |
| ------------------------------------------------- | ---------------- |
| Add extra information to your *main* dataset      | `how="left"`     |
| Keep only records that appear in both             | `how="inner"`    |
| Investigate missing patients or unmatched records | `how="outer"`    |
| Keep all rows from supplementary table            | `how="right"`    |
| Generate all combinations (rare)                  | `how="cross"`    |

---


### 💡 Teaching tip

* Always ask yourself: *“Which dataset do I want to keep completely?”* → that’s your **left** DataFrame.
* Use **left joins** most of the time in EHR work — they preserve the full clinical cohort and simply add metadata from other tables.
* Reserve **outer joins** for *data validation or reconciliation* tasks (e.g., checking which patient IDs failed to match).


### 🧪 <span style="color:red; font-weight:bold">Student Challenge</span>

> 1. Merge the `extra_info` DataFrame into `df` using a **left join**.
> 2. Merge the `clinics` DataFrame into your merged dataset using `clinic_id`.
> 3. Create a summary table showing the **average BMI** and **mean LEDD** by `region`.
> 4. Identify any patients who don’t have a matching `clinic_id` (hint: `df_merged["clinic_id"].isna()`).
> 5. (Challenge) Perform an **outer join** between `df` and `extra_info` and count how many unmatched IDs exist in each dataset.


In [ ]:
df_merged = df.merge(extra_info, on="patient_id", how="left")
print("✅ Merged patient data (left join):")
display(df_merged)


✅ Merged patient data (left join):


,patient_id,sex,date_of_birth,updrs_iii,ledd_mg,visit_date,medication,other_medication,BMI,id_last_update,...,bmi_group,ledd_band,ledd_quartile,medication_type_abbrev,updrs_iii_diag,updrs_zscore,height_cm,weight_kg,smoking_status,clinic_id
0,P01,Male,1950-05-14,22.0,300,2023-01-01,Co-Carbidopa 12.5/50,Yes,22.5,2024-06-04 23:25:00,...,normal weight,"(212.5, 425.0]","(-0.001, 325.0]",LD,No PD,-0.707107,172.0,72.0,Former,101.0
1,P02,Female,1965-08-22,30.0,500,2023-01-02,Rotigotine 0.5,Yes,25.0,2024-10-04 23:25:00,...,overweight,"(425.0, 637.5]","(325.0, 500.0]",DA,No PD,-0.707107,160.0,65.0,Never,102.0
2,P03,Female,1970-11-30,40.0,800,2023-01-03,Co-BEneldopa;,No,30.0,2023-09-03 23:25:00,...,obesity I,"(637.5, 850.0]","(587.5, 850.0]",LD,PD,NaN,158.0,60.0,Current,101.0
3,P04,Male,1980-01-15,NaN,0,2023-01-04,,No,28.0,2024-06-04 23:25:00,...,obesity I,"(-0.85, 212.5]","(-0.001, 325.0]",None,NaN,NaN,182.0,80.0,Never,103.0
4,P01,Male,1950-05-14,25.0,500,2024-02-08,"Co-carbidopa,,",Yes,22.7,2024-06-04 23:25:00,...,normal weight,"(425.0, 637.5]","(325.0, 500.0]",LD,No PD,0.707107,172.0,72.0,Former,101.0
5,P02,Female,1965-08-22,31.0,600,2025-02-23,Rotigotine.,No,24.0,2024-10-04 23:25:00,...,normal weight,"(425.0, 637.5]","(587.5, 850.0]",DA,No PD,0.707107,160.0,65.0,Never,102.0
6,P03,Female,1970-11-30,40.0,850,2023-10-06,LEVODOPA + ENTACAPONE,Yes,36.0,2023-09-03 23:25:00,...,obesity III,"(637.5, 850.0]","(587.5, 850.0]",COMT+LDR,PD,NaN,158.0,60.0,Current,101.0
7,P04,Male,1980-01-15,NaN,0,2024-06-04,N/A,No,27.6,2024-06-04 23:25:00,...,obesity I,"(-0.85, 212.5]","(-0.001, 325.0]",None,NaN,NaN,182.0,80.0,Never,103.0
8,P05,Male,1960-10-10,42.0,550,2024-05-01,Entacapone 200,NaN,26.4,NaT,...,obesity I,"(425.0, 637.5]","(500.0, 587.5]",COMT,PD,NaN,NaN,NaN,NaN,NaN
9,P06,Unidentified,1980-05-22,28.0,400,2025-01-05,Levodopa,No,23.4,2025-06-04 23:25:00,...,normal weight,"(212.5, 425.0]","(325.0, 500.0]",LD,No PD,NaN,NaN,NaN,NaN,NaN


In [ ]:
df_merged = df_merged.merge(clinics, on="clinic_id", how="left")
print("✅ Added clinic information:")
display(df_merged)


✅ Added clinic information:


,patient_id,sex,date_of_birth,updrs_iii,ledd_mg,visit_date,medication,other_medication,BMI,id_last_update,...,medication_type_abbrev,updrs_iii_diag,updrs_zscore,height_cm,weight_kg,smoking_status,clinic_id,clinic_name,deprivation_index,region
0,P01,Male,1950-05-14,22.0,300,2023-01-01,Co-Carbidopa 12.5/50,Yes,22.5,2024-06-04 23:25:00,...,LD,No PD,-0.707107,172.0,72.0,Former,101.0,St. Mary’s,0.25,London
1,P02,Female,1965-08-22,30.0,500,2023-01-02,Rotigotine 0.5,Yes,25.0,2024-10-04 23:25:00,...,DA,No PD,-0.707107,160.0,65.0,Never,102.0,Queen Elizabeth,0.52,Birmingham
2,P03,Female,1970-11-30,40.0,800,2023-01-03,Co-BEneldopa;,No,30.0,2023-09-03 23:25:00,...,LD,PD,NaN,158.0,60.0,Current,101.0,St. Mary’s,0.25,London
3,P04,Male,1980-01-15,NaN,0,2023-01-04,,No,28.0,2024-06-04 23:25:00,...,None,NaN,NaN,182.0,80.0,Never,103.0,Royal Infirmary,0.41,Manchester
4,P01,Male,1950-05-14,25.0,500,2024-02-08,"Co-carbidopa,,",Yes,22.7,2024-06-04 23:25:00,...,LD,No PD,0.707107,172.0,72.0,Former,101.0,St. Mary’s,0.25,London
5,P02,Female,1965-08-22,31.0,600,2025-02-23,Rotigotine.,No,24.0,2024-10-04 23:25:00,...,DA,No PD,0.707107,160.0,65.0,Never,102.0,Queen Elizabeth,0.52,Birmingham
6,P03,Female,1970-11-30,40.0,850,2023-10-06,LEVODOPA + ENTACAPONE,Yes,36.0,2023-09-03 23:25:00,...,COMT+LDR,PD,NaN,158.0,60.0,Current,101.0,St. Mary’s,0.25,London
7,P04,Male,1980-01-15,NaN,0,2024-06-04,N/A,No,27.6,2024-06-04 23:25:00,...,None,NaN,NaN,182.0,80.0,Never,103.0,Royal Infirmary,0.41,Manchester
8,P05,Male,1960-10-10,42.0,550,2024-05-01,Entacapone 200,NaN,26.4,NaT,...,COMT,PD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,P06,Unidentified,1980-05-22,28.0,400,2025-01-05,Levodopa,No,23.4,2025-06-04 23:25:00,...,LD,No PD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
summary_by_region = (
    df_merged.groupby("region")[["BMI", "ledd_mg"]]
    .mean()
    .round(1)
    .reset_index()
)
print("✅ Average BMI and mean LEDD by region:")
display(summary_by_region)


✅ Average BMI and mean LEDD by region:


,region,BMI,ledd_mg
0,Birmingham,24.5,550.0
1,London,27.8,612.5
2,Manchester,27.8,0.0


In [ ]:
missing_clinic = df_merged[df_merged["clinic_id"].isna()]
print("⚠️ Patients without matching clinic info:")
display(missing_clinic[["patient_id", "clinic_id"]])


⚠️ Patients without matching clinic info:


,patient_id,clinic_id
8,P05,NaN
9,P06,NaN


In [ ]:
df_outer = df.merge(extra_info, on="patient_id", how="outer", indicator=True)
print("✅ Outer join results:")
display(df_outer)

# Count unmatched rows by join source
join_counts = df_outer["_merge"].value_counts()
print("\n🔍 Where did each row come from?")
display(join_counts)


✅ Outer join results:


,patient_id,sex,date_of_birth,updrs_iii,ledd_mg,visit_date,medication,other_medication,BMI,id_last_update,...,ledd_band,ledd_quartile,medication_type_abbrev,updrs_iii_diag,updrs_zscore,height_cm,weight_kg,smoking_status,clinic_id,_merge
0,P01,Male,1950-05-14,22.0,300.0,2023-01-01,Co-Carbidopa 12.5/50,Yes,22.5,2024-06-04 23:25:00,...,"(212.5, 425.0]","(-0.001, 325.0]",LD,No PD,-0.707107,172.0,72.0,Former,101.0,both
1,P01,Male,1950-05-14,25.0,500.0,2024-02-08,"Co-carbidopa,,",Yes,22.7,2024-06-04 23:25:00,...,"(425.0, 637.5]","(325.0, 500.0]",LD,No PD,0.707107,172.0,72.0,Former,101.0,both
2,P02,Female,1965-08-22,30.0,500.0,2023-01-02,Rotigotine 0.5,Yes,25.0,2024-10-04 23:25:00,...,"(425.0, 637.5]","(325.0, 500.0]",DA,No PD,-0.707107,160.0,65.0,Never,102.0,both
3,P02,Female,1965-08-22,31.0,600.0,2025-02-23,Rotigotine.,No,24.0,2024-10-04 23:25:00,...,"(425.0, 637.5]","(587.5, 850.0]",DA,No PD,0.707107,160.0,65.0,Never,102.0,both
4,P03,Female,1970-11-30,40.0,800.0,2023-01-03,Co-BEneldopa;,No,30.0,2023-09-03 23:25:00,...,"(637.5, 850.0]","(587.5, 850.0]",LD,PD,NaN,158.0,60.0,Current,101.0,both
5,P03,Female,1970-11-30,40.0,850.0,2023-10-06,LEVODOPA + ENTACAPONE,Yes,36.0,2023-09-03 23:25:00,...,"(637.5, 850.0]","(587.5, 850.0]",COMT+LDR,PD,NaN,158.0,60.0,Current,101.0,both
6,P04,Male,1980-01-15,NaN,0.0,2023-01-04,,No,28.0,2024-06-04 23:25:00,...,"(-0.85, 212.5]","(-0.001, 325.0]",None,NaN,NaN,182.0,80.0,Never,103.0,both
7,P04,Male,1980-01-15,NaN,0.0,2024-06-04,N/A,No,27.6,2024-06-04 23:25:00,...,"(-0.85, 212.5]","(-0.001, 325.0]",None,NaN,NaN,182.0,80.0,Never,103.0,both
8,P05,Male,1960-10-10,42.0,550.0,2024-05-01,Entacapone 200,NaN,26.4,NaT,...,"(425.0, 637.5]","(500.0, 587.5]",COMT,PD,NaN,NaN,NaN,NaN,NaN,left_only
9,P06,Unidentified,1980-05-22,28.0,400.0,2025-01-05,Levodopa,No,23.4,2025-06-04 23:25:00,...,"(212.5, 425.0]","(325.0, 500.0]",LD,No PD,NaN,NaN,NaN,NaN,NaN,left_only



🔍 Where did each row come from?


_merge
both          8
left_only     2
right_only    1
Name: count, dtype: int64